In [1]:
# importando bibliotecas conexão PDGT
import json
import argparse
import subprocess
import boto3
import time
from pyathena import connect
import pandas.io.sql as sqlio
import sys
from ydata_profiling import ProfileReport
from botocore import UNSIGNED
from botocore.config import Config
import boto3.session
from botocore import exceptions
from scipy.stats import chi2_contingency

/home/lucasmateus/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# importando bibliotecas para análise
import pandas as pd
from scipy import stats
import math
import statsmodels.stats.api as sms
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import psycopg2
import dask.dataframe as dd
from collections import defaultdict
from scipy.stats import shapiro
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.stats import mode
from scipy.stats import f_oneway
from sklearn.model_selection import train_test_split
import googlemaps
from datetime import datetime
from googlemaps.exceptions import ApiError

In [3]:
class CustomException(Exception):
    pass

In [4]:
json_manifest_dbt = "target/manifest.json"
athena_bucket = "todos-athena-us-east-1"
athena_tmp_folder = f"s3://{athena_bucket}/"

In [5]:
def execute_athena_sql(query):
    client = boto3.client('athena', region_name='us-east-1')
    queryStart = client.start_query_execution(
    QueryString = query,
    ResultConfiguration = { 'OutputLocation': athena_tmp_folder})
    queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])
    while queryExecution['QueryExecution']['Status']['State'] in ('RUNNING', 'QUEUED'):
        time.sleep(5)
        queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])

In [6]:
def execute_athena_query(query):
    cursor = connect(s3_staging_dir=athena_tmp_folder,
                    region_name="us-east-1").cursor()
    cursor.execute(query)
    colls=','.join(str(f"{e[0]}") for e in cursor.description)
    results=pd.DataFrame(list(cursor), columns=colls.split(","))
    return results

In [7]:
# SETANDO API DO MAPS

#api_key = "token"

gmaps_client = googlemaps.Client(key=api_key)

NameError: name 'api_key' is not defined

In [ ]:
query_proporcoes = """
WITH regional_mais_utilizada AS (
	WITH regional_paciente_homem AS (
	    SELECT
	        cr.id_paciente,
	        cr.regional,
	        --COUNT(*) AS qtd_por_regional
	        ROW_NUMBER() OVER(PARTITION BY cr.id_paciente ORDER BY COUNT(*) DESC, cr.regional) AS rn
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON 
	    	pe.paciente_id = cr.id_paciente
	    LEFT JOIN todos_data_lake_trusted_feegow.pacientes pc ON 
	    	pc.id = cr.id_paciente
	    WHERE 1=1
	    AND pc.sexo = 1
	    AND pc.sys_active = 1
	    AND cr.cpfpaciente IS NOT NULL
	    AND cr.cpfpaciente <> ''
	    AND cr.regional IS NOT NULL
	    AND cr.nomegrupo in ('Consultas', 'Sessão', 'Exames Laboratoriais', 'Procedimentos' , 'Exames de Imagem')
	    AND pc.nascimento IS NOT NULL
		AND pe.logradouro IS NOT NULL
		AND pe.numero IS NOT NULL
		AND pe.cidade IS NOT NULL
		AND pe.estado IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 
	'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
		AND DATE_DIFF('year', pc.nascimento, current_date) BETWEEN 0 AND 100
	    GROUP BY
	        cr.id_paciente,
	        cr.regional
    ),
   regional_paciente_mulher AS (
	    SELECT
	        cr.id_paciente,
	        cr.regional,
	        --COUNT(*) AS qtd_por_regional
	        ROW_NUMBER() OVER(PARTITION BY cr.id_paciente ORDER BY COUNT(*) DESC, cr.regional) AS rn
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON 
	    	pe.paciente_id = cr.id_paciente
	    LEFT JOIN todos_data_lake_trusted_feegow.pacientes pc ON 
	    	pc.id = cr.id_paciente
	    WHERE 1=1
	    AND pc.sexo = 2
	    AND pc.sys_active = 1
	    AND cr.cpfpaciente IS NOT NULL
	    AND cr.cpfpaciente <> ''
	    AND cr.regional IS NOT NULL
	    AND cr.nomegrupo in ('Consultas', 'Sessão', 'Exames Laboratoriais', 'Procedimentos' , 'Exames de Imagem')
	    AND pc.nascimento IS NOT NULL
		AND pe.logradouro IS NOT NULL
		AND pe.numero IS NOT NULL
		AND pe.cidade IS NOT NULL
		AND pe.estado IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 
	'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
		AND DATE_DIFF('year', pc.nascimento, current_date) BETWEEN 0 AND 100
	    GROUP BY
	        cr.id_paciente,
	        cr.regional
    )
SELECT DISTINCT
    rp.id_paciente,
    rp.regional,
    1 AS sexo
FROM regional_paciente_homem rp
WHERE rp.rn = 1
UNION ALL 
SELECT DISTINCT
    rp.id_paciente,
    rp.regional,
    2 AS sexo
FROM regional_paciente_mulher rp
WHERE rp.rn = 1
),
total_pacientes AS (
    SELECT COUNT(DISTINCT id_paciente) AS total_pacientes
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
    LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON 
	    	pe.paciente_id = cr.id_paciente
    LEFT JOIN todos_data_lake_trusted_feegow.pacientes pc ON 
    	pc.id = cr.id_paciente
    WHERE 1=1
    AND pc.sexo IN (1,2)
    AND pc.sys_active = 1
    AND cr.cpfpaciente IS NOT NULL
    AND cr.cpfpaciente <> ''
    AND cr.regional IS NOT NULL
    AND cr.nomegrupo in ('Consultas', 'Sessão', 'Exames Laboratoriais', 'Procedimentos' , 'Exames de Imagem')
    AND pc.nascimento IS NOT NULL
	AND pe.logradouro IS NOT NULL
	AND pe.numero IS NOT NULL
	AND pe.cidade IS NOT NULL
	AND pe.estado IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 
'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
	AND DATE_DIFF('year', pc.nascimento, current_date) BETWEEN 0 AND 100
)
SELECT 
	rmu.regional,
	SUM(CASE WHEN rmu.sexo = 1 THEN 1 ELSE NULL END) AS  qtd_pacientes_homens,
	SUM(CASE WHEN rmu.sexo = 2 THEN 1 ELSE NULL END) AS qtd_pacientes_mulheres,
	tp.total_pacientes
FROM regional_mais_utilizada rmu
CROSS JOIN total_pacientes tp
GROUP BY 
	rmu.regional,
	tp.total_pacientes
ORDER BY count(rmu.id_paciente)
"""

In [ ]:
df_proporcoes = execute_athena_query(query_proporcoes)

In [ ]:
df_proporcoes['proporcao_homens'] = df_proporcoes['qtd_pacientes_homens'] / df_proporcoes['total_pacientes']
df_proporcoes['proporcao_mulheres'] = df_proporcoes['qtd_pacientes_mulheres'] / df_proporcoes['total_pacientes']

df_proporcoes

,regional,qtd_pacientes_homens,qtd_pacientes_mulheres,total_pacientes,proporcao_homens,proporcao_mulheres
0,ES,107660,196503,7612762,0.014142,0.025812
1,Norte,135914,250751,7612762,0.017853,0.032938
2,NE2,180588,362902,7612762,0.023722,0.047670
3,RJ,216168,404848,7612762,0.028395,0.053180
4,CO,231395,428580,7612762,0.030396,0.056298
5,MG,268464,485084,7612762,0.035265,0.063720
6,NE1,293109,564595,7612762,0.038502,0.074164
7,Sul,378680,669421,7612762,0.049743,0.087934
8,SP Interior,440010,748676,7612762,0.057799,0.098345
9,SP CAV,438208,811206,7612762,0.057562,0.106559


In [ ]:
#COCHRAN FORMULA FOR SAMPLE SIZE - POPULAÇÃO COMPLETA

def cochran_sample_size(confidence, estimated_proportion, margin_of_error):
    z = np.abs(stats.norm.ppf((1 - confidence) / 2))  # Calculate Z-score
    n = (z**2 * estimated_proportion * (1 - estimated_proportion)) / margin_of_error**2
    return n

# Example values
confidence_level = 0.99  # 99% confidence
estimated_proportion = 0.5  # Estimated proportion (50% for maximum sample size)
margin_of_error = 0.01  # Margin of error (1%)

# Calculate sample size
sample_size = cochran_sample_size(confidence_level, estimated_proportion, margin_of_error)

print(f"Required Sample Size: {int(sample_size)}")

Required Sample Size: 16587


In [ ]:
df_proporcoes['tamanho_amostra'] = sample_size
df_proporcoes['amostra_homens'] = (df_proporcoes['proporcao_homens'] * sample_size).apply(lambda x: math.ceil(x))
df_proporcoes['amostra_mulheres'] = (df_proporcoes['proporcao_mulheres'] * sample_size).apply(lambda x: math.ceil(x))

df_proporcoes

,regional,qtd_pacientes_homens,qtd_pacientes_mulheres,total_pacientes,proporcao_homens,proporcao_mulheres,tamanho_amostra,amostra_homens,amostra_mulheres
0,ES,107660,196503,7612762,0.014142,0.025812,16587.241503,235,429
1,Norte,135914,250751,7612762,0.017853,0.032938,16587.241503,297,547
2,NE2,180588,362902,7612762,0.023722,0.047670,16587.241503,394,791
3,RJ,216168,404848,7612762,0.028395,0.053180,16587.241503,472,883
4,CO,231395,428580,7612762,0.030396,0.056298,16587.241503,505,934
5,MG,268464,485084,7612762,0.035265,0.063720,16587.241503,585,1057
6,NE1,293109,564595,7612762,0.038502,0.074164,16587.241503,639,1231
7,Sul,378680,669421,7612762,0.049743,0.087934,16587.241503,826,1459
8,SP Interior,440010,748676,7612762,0.057799,0.098345,16587.241503,959,1632
9,SP CAV,438208,811206,7612762,0.057562,0.106559,16587.241503,955,1768


In [ ]:
#TRANSFORMANDO VALORES DA TABELA EM VARIÁVEIS - HOMENS

spcav_h = df_proporcoes.loc[df_proporcoes['regional'] == 'SP CAV', 'amostra_homens'].values[0]
spint_h = df_proporcoes.loc[df_proporcoes['regional'] == 'SP Interior', 'amostra_homens'].values[0]
sul_h = df_proporcoes.loc[df_proporcoes['regional'] == 'Sul', 'amostra_homens'].values[0]
ne1_h = df_proporcoes.loc[df_proporcoes['regional'] == 'NE1', 'amostra_homens'].values[0]
mg_h = df_proporcoes.loc[df_proporcoes['regional'] == 'MG', 'amostra_homens'].values[0]
co_h = df_proporcoes.loc[df_proporcoes['regional'] == 'CO', 'amostra_homens'].values[0]
rj_h = df_proporcoes.loc[df_proporcoes['regional'] == 'RJ', 'amostra_homens'].values[0]
ne2_h = df_proporcoes.loc[df_proporcoes['regional'] == 'NE2', 'amostra_homens'].values[0]
norte_h = df_proporcoes.loc[df_proporcoes['regional'] == 'Norte', 'amostra_homens'].values[0]
es_h = df_proporcoes.loc[df_proporcoes['regional'] == 'ES', 'amostra_homens'].values[0]

print(f'Amostra homens SP CAV: {spcav_h}')
print(f'Amostra homens SP Int: {spint_h}')
print(f'Amostra homens Sul: {sul_h}')
print(f'Amostra homens NE1: {ne1_h}')
print(f'Amostra homens MG: {mg_h}')
print(f'Amostra homens CO: {co_h}')
print(f'Amostra homens RJ: {rj_h}')
print(f'Amostra homens NE2: {ne2_h}')
print(f'Amostra homens Norte: {norte_h}')
print(f'Amostra homens ES: {es_h}')

Amostra homens SP CAV: 955
Amostra homens SP Int: 959
Amostra homens Sul: 826
Amostra homens NE1: 639
Amostra homens MG: 585
Amostra homens CO: 505
Amostra homens RJ: 472
Amostra homens NE2: 394
Amostra homens Norte: 297
Amostra homens ES: 235


In [ ]:
#TRANSFORMANDO VALORES DA TABELA EM VARIÁVEIS - MULHERES

spcav_m = df_proporcoes.loc[df_proporcoes['regional'] == 'SP CAV', 'amostra_mulheres'].values[0]
spint_m = df_proporcoes.loc[df_proporcoes['regional'] == 'SP Interior', 'amostra_mulheres'].values[0]
sul_m = df_proporcoes.loc[df_proporcoes['regional'] == 'Sul', 'amostra_mulheres'].values[0]
ne1_m = df_proporcoes.loc[df_proporcoes['regional'] == 'NE1', 'amostra_mulheres'].values[0]
mg_m = df_proporcoes.loc[df_proporcoes['regional'] == 'MG', 'amostra_mulheres'].values[0]
co_m = df_proporcoes.loc[df_proporcoes['regional'] == 'CO', 'amostra_mulheres'].values[0]
rj_m = df_proporcoes.loc[df_proporcoes['regional'] == 'RJ', 'amostra_mulheres'].values[0]
ne2_m = df_proporcoes.loc[df_proporcoes['regional'] == 'NE2', 'amostra_mulheres'].values[0]
norte_m = df_proporcoes.loc[df_proporcoes['regional'] == 'Norte', 'amostra_mulheres'].values[0]
es_m = df_proporcoes.loc[df_proporcoes['regional'] == 'ES', 'amostra_mulheres'].values[0]

print(f'Amostra mulheres SP CAV: {spcav_m}')
print(f'Amostra mulheres SP Int: {spint_m}')
print(f'Amostra mulheres Sul: {sul_m}')
print(f'Amostra mulheres NE1: {ne1_m}')
print(f'Amostra mulheres MG: {mg_m}')
print(f'Amostra mulheres CO: {co_m}')
print(f'Amostra mulheres RJ: {rj_m}')
print(f'Amostra mulheres NE2: {ne2_m}')
print(f'Amostra mulheres Norte: {norte_m}')
print(f'Amostra hommulheresens ES: {es_m}')

Amostra mulheres SP CAV: 1768
Amostra mulheres SP Int: 1632
Amostra mulheres Sul: 1459
Amostra mulheres NE1: 1231
Amostra mulheres MG: 1057
Amostra mulheres CO: 934
Amostra mulheres RJ: 883
Amostra mulheres NE2: 791
Amostra mulheres Norte: 547
Amostra hommulheresens ES: 429


In [19]:
#COCHRAN FORMULA FOR SAMPLE SIZE - PACIENTE PARTICULAR

def cochran_sample_size(confidence, estimated_proportion, margin_of_error):
    z = np.abs(stats.norm.ppf((1 - confidence) / 2))  # Calculate Z-score
    n = (z**2 * estimated_proportion * (1 - estimated_proportion)) / margin_of_error**2
    return n

# Example values
confidence_level = 0.90  # 99% confidence
estimated_proportion = 0.05  # Estimated proportion (50% for maximum sample size)
margin_of_error = 0.05  # Margin of error (1%)

# Calculate sample size
sample_size_part = cochran_sample_size(confidence_level, estimated_proportion, margin_of_error)

print(f"Tamanho da amostra necessária: {int(sample_size_part)} para {confidence_level*100:.0f}% de nível de confiança e {margin_of_error*100:.0f}% de margem de erro.")



Tamanho da amostra necessária: 270 para 90% de nível de confiança e 5% de margem de erro.


In [ ]:
df_proporcoes['tamanho_amostra_part'] = sample_size_part
df_proporcoes['amostra_homens_part'] = (df_proporcoes['proporcao_homens'] * sample_size_part).apply(lambda x: math.ceil(x))
df_proporcoes['amostra_mulheres_part'] = (df_proporcoes['proporcao_mulheres'] * sample_size_part).apply(lambda x: math.ceil(x))

df_proporcoes

,regional,qtd_pacientes_homens,qtd_pacientes_mulheres,total_pacientes,proporcao_homens,proporcao_mulheres,tamanho_amostra,amostra_homens,amostra_mulheres,tamanho_amostra_part,amostra_homens_part,amostra_mulheres_part
0,ES,107660,196503,7612762,0.014142,0.025812,16587.241503,235,429,3151.575885,45,82
1,Norte,135914,250751,7612762,0.017853,0.032938,16587.241503,297,547,3151.575885,57,104
2,NE2,180588,362902,7612762,0.023722,0.047670,16587.241503,394,791,3151.575885,75,151
3,RJ,216168,404848,7612762,0.028395,0.053180,16587.241503,472,883,3151.575885,90,168
4,CO,231395,428580,7612762,0.030396,0.056298,16587.241503,505,934,3151.575885,96,178
5,MG,268464,485084,7612762,0.035265,0.063720,16587.241503,585,1057,3151.575885,112,201
6,NE1,293109,564595,7612762,0.038502,0.074164,16587.241503,639,1231,3151.575885,122,234
7,Sul,378680,669421,7612762,0.049743,0.087934,16587.241503,826,1459,3151.575885,157,278
8,SP Interior,440010,748676,7612762,0.057799,0.098345,16587.241503,959,1632,3151.575885,183,310
9,SP CAV,438208,811206,7612762,0.057562,0.106559,16587.241503,955,1768,3151.575885,182,336


In [ ]:
#TRANSFORMANDO VALORES DA TABELA EM VARIÁVEIS - HOMENS

spcav_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'SP CAV', 'amostra_homens_part'].values[0]
spint_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'SP Interior', 'amostra_homens_part'].values[0]
sul_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'Sul', 'amostra_homens_part'].values[0]
ne1_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'NE1', 'amostra_homens_part'].values[0]
mg_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'MG', 'amostra_homens_part'].values[0]
co_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'CO', 'amostra_homens_part'].values[0]
rj_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'RJ', 'amostra_homens_part'].values[0]
ne2_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'NE2', 'amostra_homens_part'].values[0]
norte_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'Norte', 'amostra_homens_part'].values[0]
es_h_part = df_proporcoes.loc[df_proporcoes['regional'] == 'ES', 'amostra_homens_part'].values[0]

print(f'Amostra homens SP CAV: {spcav_h_part}')
print(f'Amostra homens SP Int: {spint_h_part}')
print(f'Amostra homens Sul: {sul_h_part}')
print(f'Amostra homens NE1: {ne1_h_part}')
print(f'Amostra homens MG: {mg_h_part}')
print(f'Amostra homens CO: {co_h_part}')
print(f'Amostra homens RJ: {rj_h_part}')
print(f'Amostra homens NE2: {ne2_h_part}')
print(f'Amostra homens Norte: {norte_h_part}')
print(f'Amostra homens ES: {es_h_part}')

Amostra homens SP CAV: 182
Amostra homens SP Int: 183
Amostra homens Sul: 157
Amostra homens NE1: 122
Amostra homens MG: 112
Amostra homens CO: 96
Amostra homens RJ: 90
Amostra homens NE2: 75
Amostra homens Norte: 57
Amostra homens ES: 45


In [ ]:
#TRANSFORMANDO VALORES DA TABELA EM VARIÁVEIS - MULHERES

spcav_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'SP CAV', 'amostra_mulheres_part'].values[0]
spint_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'SP Interior', 'amostra_mulheres_part'].values[0]
sul_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'Sul', 'amostra_mulheres_part'].values[0]
ne1_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'NE1', 'amostra_mulheres_part'].values[0]
mg_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'MG', 'amostra_mulheres_part'].values[0]
co_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'CO', 'amostra_mulheres_part'].values[0]
rj_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'RJ', 'amostra_mulheres_part'].values[0]
ne2_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'NE2', 'amostra_mulheres_part'].values[0]
norte_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'Norte', 'amostra_mulheres_part'].values[0]
es_m_part = df_proporcoes.loc[df_proporcoes['regional'] == 'ES', 'amostra_mulheres_part'].values[0]

print(f'Amostra mulheres SP CAV: {spcav_m_part}')
print(f'Amostra mulheres SP Int: {spint_m_part}')
print(f'Amostra mulheres Sul: {sul_m_part}')
print(f'Amostra mulheres NE1: {ne1_m_part}')
print(f'Amostra mulheres MG: {mg_m_part}')
print(f'Amostra mulheres CO: {co_m_part}')
print(f'Amostra mulheres RJ: {rj_m_part}')
print(f'Amostra mulheres NE2: {ne2_m_part}')
print(f'Amostra mulheres Norte: {norte_m_part}')
print(f'Amostra hommulheresens ES: {es_m_part}')

Amostra mulheres SP CAV: 336
Amostra mulheres SP Int: 310
Amostra mulheres Sul: 278
Amostra mulheres NE1: 234
Amostra mulheres MG: 201
Amostra mulheres CO: 178
Amostra mulheres RJ: 168
Amostra mulheres NE2: 151
Amostra mulheres Norte: 104
Amostra hommulheresens ES: 82


In [ ]:
query_pacientes = f"""
--BASE PACIENTES
--SELECT count(*) from (
WITH grouped_pacientes AS (
    SELECT
        cr.id_paciente,
        cr.cpfpaciente,
        MIN(cr.datapagamento) AS min_dt_pg,
        MAX(cr.datapagamento) AS max_dt_pg,
        DATE_DIFF('year', MIN(cr.datapagamento), current_date) AS anos_utilizacao,
        DATE_DIFF('month', MIN(cr.datapagamento), current_date) AS meses_utilizacao,
        DATE_DIFF('day', MAX(cr.datapagamento), current_date) AS dias_ult_utilizacao,
        DATE_DIFF('month', MAX(cr.datapagamento), current_date) AS meses_ult_utilizacao,
        UPPER(pe.estado) AS estado,
    	UPPER(pe.cidade) AS cidade,
    	DATE_DIFF('year', pc.nascimento, current_date) AS idade,
    	sx.nomesexo AS sexo,
    	upper(concat(regexp_replace(regexp_replace(pe.logradouro, '[0-9]', ''), ', ', ''), ', ', pe.numero, ', ', pe.cidade, ', ', pe.estado)) endereco_paciente
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr 
    LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON pe.paciente_id = cr.id_paciente
	LEFT JOIN todos_data_lake_trusted_feegow.pacientes pc ON pc.id = cr.id_paciente
	LEFT JOIN todos_data_lake_trusted_feegow.sexo sx ON sx.id = pc.sexo
    WHERE 1=1
    	AND pc.sexo IN (1,2)
    	AND pc.sys_active = 1
	    AND pc.nascimento IS NOT NULL
		AND pe.logradouro IS NOT NULL
		AND pe.numero IS NOT NULL
		AND pe.cidade IS NOT NULL
	    --AND pc.email IS NOT NULL
	    --AND pc.email <> ''
		AND cr.cpfpaciente IS NOT NULL
		AND cr.cpfpaciente <> ''
		AND pc.nascimento IS NOT NULL
		AND pe.estado IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 
		'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
    GROUP BY
        cr.id_paciente,
        cr.cpfpaciente,
        pe.estado,
        pe.cidade,
        pe.bairro,
        pc.nascimento,
        sx.nomesexo,
        pe.logradouro,
        pe.numero,
        pe.cidade,
        pe.estado
),
consultas AS (
    SELECT
        cr.id_paciente,
        COUNT(*) AS qtd_consultas,
        SUM(cr.valor_pago) AS tt_consulta,
        AVG(cr.valor_pago) AS tm_consulta
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
    WHERE 1=1
    	AND cr.nomegrupo in ('Consultas', 'Sessão')
    GROUP BY
        cr.id_paciente
 ),
orcamento_nao_executado AS (
    SELECT
        pp.pacienteid AS id_paciente,
        COUNT(*) AS qtd_orc_nao_executado,
        SUM(pp.valor) AS tt_orc_nao_executado
    FROM todos_data_lake_trusted_feegow.propostas pp
    WHERE 1=1
        AND pp.staid NOT IN (5, 2)
    GROUP BY
        pp.pacienteid
 ), 
particular AS (
    SELECT
        cr.id_paciente,
        COUNT(*) AS qtd_particular
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
    WHERE 1=1
        AND cr.nome_tabela_particular = 'PARTICULAR*'
        AND cr.nomegrupo in ('Consultas', 'Sessão')
    GROUP BY
        cr.id_paciente
 ), 
exames_procedimentos AS (
    SELECT
        base.id_paciente,
        SUM(base.tt_exam_proc_data) tt_exam_proc,
        SUM(base.visitas) AS qtd_exam_proc
    FROM (
        SELECT
            cr.id_paciente,
            cr.datapagamento,
            SUM(cr.valor_pago) AS tt_exam_proc_data,
            1 AS visitas
        FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
        WHERE 1=1
        AND cr.nomegrupo IN ('Exames Laboratoriais', 'Procedimentos' , 'Exames de Imagem')
        GROUP BY
            cr.id_paciente, 
            cr.datapagamento
    ) base
    GROUP BY
        base.id_paciente
), 
canal AS (
	WITH canal_paciente AS (
	    SELECT
	        ag.id_paciente,
	        CASE WHEN ag.nm_canal = 'Clínica' THEN ag.nm_subcanal 
	        	 WHEN ag.nm_canal = 'App Cartão de Todos' THEN 'Aplicativo'
	        	 WHEN ag.nm_canal IN ('Site AmorSaúde', 'LabNine', 'Tutti', 'WhatsApp', 'Telefone', 'Chat') THEN 'Outros'
	        	 ELSE ag.nm_canal END nm_canal,
	        count(*) qtd_canais,
	        ROW_NUMBER() OVER(PARTITION BY ag.id_paciente ORDER BY max(ag.dt_agendamento) DESC, ag.nm_canal) AS rn
	    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
	    WHERE 1=1
	    	AND ag.nm_canal IS NOT NULL
	    	AND ag.tipoprocedimento = 'Consulta'
	    GROUP BY
	        ag.id_paciente,
	        ag.nm_canal,
	        ag.nm_subcanal
    )
SELECT 
    cp.id_paciente,
    COALESCE(cp.nm_canal, 'Não informado') AS nm_canal,
    cp.qtd_canais,
    CASE WHEN cp.qtd_canais > 1 THEN 1
    	 ELSE 0 END mudou_canal
FROM canal_paciente cp
WHERE cp.rn = 1
),
localidade AS (
	WITH regional_unidade_paciente AS (
	    SELECT
	        cr.id_paciente,
	        cr.regional,
	        cr.id_unidade,
	        cr.nome_unidade,
	        ROW_NUMBER() OVER(PARTITION BY cr.id_paciente ORDER BY COUNT(*) DESC, cr.regional) AS rn
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    --LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON 
	    	--pe.paciente_id = cr.id_paciente
	    WHERE 1=1
	    	AND cr.regional IS NOT NULL
	    	AND cr.id_unidade <> 19774
	    	AND cr.nome_unidade IS NOT NULL
	    GROUP BY
	        cr.id_paciente,
	        cr.regional,
	        cr.id_unidade,
	        cr.nome_unidade
    )
SELECT 
    rup.id_paciente,
    rup.id_unidade,
    rup.nome_unidade AS unidade,
    upper(concat(COALESCE(un.endereco, ' '), ', ', COALESCE(un.numero, ' '), ', ', COALESCE(un.cidade, ' '), ', ', COALESCE(un.estado, ' '))) AS endereco_unidade,
    upper(un.cidade) AS cidade_clinica,
    COALESCE(rup.regional, 'Não informado') AS regional
FROM regional_unidade_paciente rup
LEFT JOIN todos_data_lake_trusted_feegow.unidades un ON rup.id_unidade = un.id
WHERE rup.rn = 1
),
especialidade_mais_utilizada AS (
	WITH especialidade_paciente AS (
	    SELECT
	        ag.id_paciente,
	        ag.nm_especialidade,
	        ROW_NUMBER() OVER(PARTITION BY ag.id_paciente ORDER BY COUNT(*) DESC, ag.nm_especialidade) AS rn
	    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
	    WHERE 1=1
	    	AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
	    	AND ag.nm_especialidade IS NOT NULL 
	    GROUP BY
	        ag.id_paciente,
	        ag.nm_especialidade
    )
SELECT 
    ep.id_paciente,
    COALESCE(ep.nm_especialidade, 'Não informado') nm_especialidade
FROM especialidade_paciente ep
WHERE ep.rn = 1
),
tabela_mais_utilizada AS (
	WITH tabela_paciente AS (
	    SELECT
	    	bd.id_paciente,
	    	bd.tabela,
	    	ROW_NUMBER() OVER(PARTITION BY bd.id_paciente ORDER BY COUNT(*) DESC, bd.tabela) AS rn
	    FROM (SELECT
	        cr.id_paciente,
	        CASE WHEN cr.nome_tabela_particular = 'Cartão de TODOS*' THEN 'Cartão de TODOS'
             WHEN cr.nome_tabela_particular = 'PARTICULAR*' THEN 'Particular'
             ELSE 'Outros' END AS tabela
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    WHERE 1=1
	    	AND cr.nomegrupo in ('Consultas', 'Sessão')
	    GROUP BY
	        cr.id_paciente,
	        cr.nome_tabela_particular
    		 ) bd
    	GROUP BY
    		bd.id_paciente,
    		bd.tabela
    )
SELECT 
    tp.id_paciente,
    COALESCE(tp.tabela, 'Não informado') tabela
FROM tabela_paciente tp
WHERE tp.rn = 1
),
base AS (SELECT DISTINCT
    gp.*,
    COALESCE(cst.qtd_consultas, 0) AS qtd_consultas,
    COALESCE(expc.qtd_exam_proc, 0) AS qtd_exam_proc, 
    COALESCE(ptcl.qtd_particular, 0) AS qtd_particular,
    COALESCE(cst.tt_consulta, 0) AS tt_consulta,
    COALESCE(cst.tm_consulta, 0) AS tm_consulta,
    COALESCE(expc.tt_exam_proc, 0) AS tt_exam_proc,
    COALESCE(one.qtd_orc_nao_executado, 0) AS qtd_orc_nao_executado,
    COALESCE(one.tt_orc_nao_executado, 0) AS tt_orc_nao_executado,
    CASE WHEN COALESCE(cst.qtd_consultas, 0) <> 0 THEN (COALESCE(cst.tt_consulta, 0) + COALESCE(expc.tt_exam_proc, 0)) / cst.qtd_consultas
         ELSE NULL END AS tm_utilizacao,
    CASE WHEN cst.qtd_consultas = ptcl.qtd_particular AND cst.qtd_consultas <> 0 THEN 1
         ELSE 0 END AS pac_particular,
    ca.nm_canal AS ultimo_canal,
    ca.qtd_canais,
    ca.mudou_canal,
    loc.regional,
    emu.nm_especialidade,
    tmu.tabela,
    loc.unidade,
    loc.endereco_unidade,
    loc.cidade_clinica
FROM grouped_pacientes gp
LEFT JOIN consultas cst ON cst.id_paciente = gp.id_paciente
LEFT JOIN particular ptcl ON ptcl.id_paciente = gp.id_paciente
LEFT JOIN exames_procedimentos expc ON expc.id_paciente = gp.id_paciente
LEFT JOIN orcamento_nao_executado one ON one.id_paciente = gp.id_paciente
LEFT JOIN canal ca ON ca.id_paciente = gp.id_paciente
LEFT JOIN localidade loc ON loc.id_paciente = gp.id_paciente
LEFT JOIN especialidade_mais_utilizada emu ON emu.id_paciente = gp.id_paciente
LEFT JOIN tabela_mais_utilizada tmu ON tmu.id_paciente = gp.id_paciente
WHERE 1=1
	AND gp.idade BETWEEN 0 AND 100),
base_score AS (
SELECT 
	*,
    CASE WHEN bd.dias_ult_utilizacao > 365 THEN 1
         WHEN bd.dias_ult_utilizacao BETWEEN 181 AND 365 THEN 2
         WHEN bd.dias_ult_utilizacao BETWEEN 91 AND 180 THEN 3
         WHEN bd.dias_ult_utilizacao BETWEEN 46 AND 90 THEN 4
         WHEN bd.dias_ult_utilizacao <= 45 THEN 5
         ELSE NULL END AS score_r,
    CASE WHEN bd.qtd_consultas = 1 THEN 1
         WHEN bd.qtd_consultas = 2 THEN 2
         WHEN bd.qtd_consultas = 3 THEN 3
         WHEN bd.qtd_consultas BETWEEN 4 AND 5 THEN 4
         WHEN bd.qtd_consultas > 5 THEN 5
         ELSE NULL END AS score_f,
    CASE WHEN bd.tm_utilizacao <= 28 THEN 1
         WHEN bd.tm_utilizacao BETWEEN 28.01 AND 56 THEN 2
         WHEN bd.tm_utilizacao BETWEEN 56.01 AND 84 THEN 3
         WHEN bd.tm_utilizacao BETWEEN 84.01 AND 140 THEN 4
         WHEN bd.tm_utilizacao > 140 THEN 5
         ELSE NULL END AS score_v	
FROM base bd),
co_h AS (
SELECT
*
FROM base_score bs
WHERE 1=1
	AND bs.regional = 'CO'
	AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{co_h}
),	
co_m AS (
SELECT
*
FROM base_score bs
WHERE 1=1
	AND bs.regional = 'CO'
	AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{co_m}
),	
spcav_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP CAV'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{spcav_h}
),
spcav_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP CAV'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{spcav_m}
),
spint_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP Interior'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{spint_h}
),
spint_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP Interior'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{spint_m}
),
sul_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Sul'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{sul_h}
),
sul_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Sul'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{sul_m}
),
rj_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'RJ'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{rj_h}
),
rj_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'RJ'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{rj_m}
),
norte_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Norte'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{norte_h}
),
norte_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Norte'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{norte_m}
),
ne2_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE2'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{ne2_h}
),
ne2_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE2'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{ne2_m}
),
ne1_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE1'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{ne1_h}
),
ne1_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE1'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{ne1_m}
),
mg_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'MG'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{mg_h}
),
mg_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'MG'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{mg_m}
),
es_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'ES'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{es_h}
),
es_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'ES'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{es_m}
)
SELECT * FROM (
    SELECT * FROM co_h
    UNION ALL
    SELECT * FROM co_m
    UNION ALL
    SELECT * FROM spcav_h
    UNION ALL
    SELECT * FROM spcav_m
    UNION ALL
    SELECT * FROM spint_h
    UNION ALL
    SELECT * FROM spint_m
    UNION ALL
    SELECT * FROM sul_h
    UNION ALL
    SELECT * FROM sul_m
    UNION ALL
    SELECT * FROM rj_h
    UNION ALL
    SELECT * FROM rj_m
    UNION ALL
    SELECT * FROM norte_h
    UNION ALL
    SELECT * FROM norte_m
    UNION ALL
    SELECT * FROM ne2_h
    UNION ALL
    SELECT * FROM ne2_m
    UNION ALL
    SELECT * FROM ne1_h
    UNION ALL
    SELECT * FROM ne1_m
    UNION ALL
    SELECT * FROM mg_h
    UNION ALL
    SELECT * FROM mg_m
    UNION ALL
    SELECT * FROM es_h
    UNION ALL
    SELECT * FROM es_m
)  amostra
--)
"""

In [ ]:
# MATERIALIZACAO DA AMOSTRA DE PACIENTES EM DF

df_pac = execute_athena_query(query_pacientes)

In [ ]:
df_pac.columns

Index(['id_paciente', 'cpfpaciente', 'min_dt_pg', 'max_dt_pg',
       'anos_utilizacao', 'meses_utilizacao', 'dias_ult_utilizacao',
       'meses_ult_utilizacao', 'estado', 'cidade', 'idade', 'sexo',
       'endereco_paciente', 'qtd_consultas', 'qtd_exam_proc', 'qtd_particular',
       'tt_consulta', 'tm_consulta', 'tt_exam_proc', 'qtd_orc_nao_executado',
       'tt_orc_nao_executado', 'tm_utilizacao', 'pac_particular',
       'ultimo_canal', 'qtd_canais', 'mudou_canal', 'regional',
       'nm_especialidade', 'tabela', 'unidade', 'endereco_unidade',
       'cidade_clinica', 'score_r', 'score_f', 'score_v'],
      dtype='object')

In [ ]:
# TRATATIVAS NA BASE DE PACIENTES

# REMOVER ACENTOS DAS CIDADES 
def remove_accents(text):
    return ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])

# Apply the function to your columns
df_pac['cidade'] = df_pac['cidade'].apply(remove_accents)
df_pac['cidade_clinica'] = df_pac['cidade_clinica'].apply(remove_accents)

# APLICAR REGRA DE MESMA CIDADE
df_pac['mesma_cidade'] = (df_pac['cidade'] == df_pac['cidade_clinica']).astype(int)

# APLICAR FAIXAS DE IDADE E UTILIZACAO
age_bins = [0, 19, 24, 29, 34, 39, 44, 49, 54, 59, float('inf')]
age_labels = ['A: 0-18', 'B: 19-23', 'C: 24-28', 'D: 29-33', 'E: 34-38', 'F: 39-43', 'G: 44-48', 'H: 49-53', 'I: 54-58', 'J: 59+']

# Add a new column 'faixa_etaria' based on 'idade' column
df_pac['faixa_etaria'] = pd.cut(df_pac['idade'], bins=age_bins, labels=age_labels, right=False)

# Define the age group bins and labels based on your classification
time_bins = [0, 7, 13, 25, 37, 49, 61, float('inf')]
time_labels = ['A: 0-6 meses', 'B: 7-12 meses', 'C: 1 ano', 'D: 2 anos', 'E: 3 anos', 'F: 4 anos', 'G: 5+ anos']

# Add a new column 'faixa_etaria' based on 'idade' column
df_pac['faixa_utilizacao'] = pd.cut(df_pac['meses_utilizacao'], bins=time_bins, labels=time_labels, right=False)

In [ ]:
# Função para traduzir as condições da regra em texto
def translate_recency_score(dias_ult_utilizacao):
    if dias_ult_utilizacao <= 45:
        return '5. <= 45 dias'
    elif 46 <= dias_ult_utilizacao <= 90:
        return '4. 46-90 dias'
    elif 91 <= dias_ult_utilizacao <= 180:
        return '3. 91-180 dias'
    elif 181 <= dias_ult_utilizacao <= 365:
        return '2. 181-365 dias'
    elif dias_ult_utilizacao > 365:
        return '1. > 365 dias'
    else:
        return 'N/A'

# Aplicar a função à coluna 'dias_ult_utilizacao' e criar a nova coluna 'score_r_text'
df_pac['score_recencia'] = df_pac['dias_ult_utilizacao'].apply(translate_recency_score)

import pandas as pd

# Função para traduzir as condições da regra em texto
def translate_frequency_score(qtd_consultas):
    if qtd_consultas == 1:
        return '1. 1 consulta'
    elif qtd_consultas == 2:
        return '2. 2 consultas'
    elif qtd_consultas == 3:
        return '3. 3 consultas'
    elif 4 <= qtd_consultas <= 5:
        return '4. 4-5 consultas'
    elif qtd_consultas > 5:
        return '5. > 5 consultas'
    else:
        return 'N/A'

# Aplicar a função à coluna 'qtd_consultas' e 'tm_utilizacao' e criar a nova coluna 'score_f_text'
df_pac['score_frequencia'] = df_pac['qtd_consultas'].map(lambda x: translate_frequency_score(x))

# Função para traduzir as condições da regra em texto
def translate_value_score(tm_utilizacao):
    if tm_utilizacao is None:
        return 'N/A'
    elif tm_utilizacao <= 28:
        return '1. <= R$ 28'
    elif 28.01 <= tm_utilizacao <= 56:
        return '2. R$ 28.01 - R$ 56'
    elif 56.01 <= tm_utilizacao <= 84:
        return '3. R$ 56.01 - R$ 84'
    elif 84.01 <= tm_utilizacao <= 140:
        return '4. R$ 84.01 - R$ 140'
    elif tm_utilizacao > 140:
        return '5. > R$ 140'
    else:
        return 'N/A'

# Aplicar a função à coluna 'tm_utilizacao' e criar a nova coluna 'score_v_text'
df_pac['score_valor'] = df_pac['tm_utilizacao'].map(lambda x: translate_value_score(x))

In [ ]:
# Iterate through the DataFrame
for index, row in df_pac.iterrows():
    source = row['endereco_paciente']
    destination = row['endereco_unidade']

    try:
        if source and destination:
            direction_result = gmaps_client.directions(source, destination)

            # Extract the distance and duration values
            if direction_result:
                legs = direction_result[0]['legs'][0]
                distance_text = legs['distance']['text']
                distance_value = legs['distance']['value']
                duration_text = legs['duration']['text']
                duration_value = legs['duration']['value']

                # You can then use these values as needed, for example, store them in your DataFrame
                df_pac.at[index, 'distance_text'] = distance_text
                df_pac.at[index, 'distance_value'] = distance_value
                df_pac.at[index, 'duration_text'] = duration_text
                df_pac.at[index, 'duration_value'] = duration_value

    except ApiError as e:
        if "NOT_FOUND" in str(e):
            # Handle the "NOT_FOUND" error (e.g., skip to the next row)
            print(f"Skipping row {index} due to 'NOT_FOUND' error.")
            continue

Skipping row 468 due to 'NOT_FOUND' error.
Skipping row 1933 due to 'NOT_FOUND' error.
Skipping row 1971 due to 'NOT_FOUND' error.
Skipping row 2094 due to 'NOT_FOUND' error.
Skipping row 2290 due to 'NOT_FOUND' error.
Skipping row 2325 due to 'NOT_FOUND' error.
Skipping row 2326 due to 'NOT_FOUND' error.
Skipping row 4572 due to 'NOT_FOUND' error.
Skipping row 4783 due to 'NOT_FOUND' error.
Skipping row 4853 due to 'NOT_FOUND' error.
Skipping row 4867 due to 'NOT_FOUND' error.
Skipping row 5460 due to 'NOT_FOUND' error.
Skipping row 5682 due to 'NOT_FOUND' error.
Skipping row 6553 due to 'NOT_FOUND' error.
Skipping row 6554 due to 'NOT_FOUND' error.
Skipping row 6671 due to 'NOT_FOUND' error.
Skipping row 8734 due to 'NOT_FOUND' error.
Skipping row 10166 due to 'NOT_FOUND' error.
Skipping row 10663 due to 'NOT_FOUND' error.
Skipping row 10755 due to 'NOT_FOUND' error.
Skipping row 10927 due to 'NOT_FOUND' error.
Skipping row 11147 due to 'NOT_FOUND' error.
Skipping row 11352 due to 'N

In [ ]:
# MOSTRAR DF

df_pac

,id_paciente,cpfpaciente,min_dt_pg,max_dt_pg,anos_utilizacao,meses_utilizacao,dias_ult_utilizacao,meses_ult_utilizacao,estado,cidade,...,mesma_cidade,faixa_etaria,faixa_utilizacao,distance_text,distance_value,duration_text,duration_value,score_recencia,score_frequencia,score_valor
0,13226290,06214612533,2023-09-11,2023-09-11,0.0,1.0,32.0,1.0,SP,GUARULHOS,...,1,D: 29-33,A: 0-6 meses,2.0 km,1951.0,7 mins,431.0,5. <= 45 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
1,14463025,16907844826,2021-09-27,2023-05-30,2.0,24.0,136.0,4.0,SP,MOGI DAS CRUZES,...,1,H: 49-53,C: 1 ano,8.0 km,7966.0,18 mins,1101.0,3. 91-180 dias,5. > 5 consultas,4. R$ 84.01 - R$ 140
2,67282839,08800649866,2023-10-05,2023-10-05,0.0,0.0,8.0,0.0,SP,BARUERI,...,1,I: 54-58,A: 0-6 meses,7.8 km,7836.0,15 mins,913.0,5. <= 45 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
3,58708224,30062835890,2021-07-27,2021-07-27,2.0,26.0,808.0,26.0,SP,ITAPECERICA DA SERRA,...,1,F: 39-43,D: 2 anos,1.4 km,1369.0,5 mins,271.0,1. > 365 dias,1. 1 consulta,1. <= R$ 28
4,58356133,25331036830,2021-06-16,2021-06-23,2.0,27.0,842.0,27.0,SP,SAO PAULO,...,1,G: 44-48,D: 2 anos,4.2 km,4196.0,11 mins,666.0,1. > 365 dias,2. 2 consultas,2. R$ 28.01 - R$ 56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16593,59362136,02898153478,2021-10-06,2023-10-11,2.0,24.0,2.0,0.0,AL,MACEIO,...,1,G: 44-48,C: 1 ano,0.9 km,902.0,3 mins,183.0,5. <= 45 dias,5. > 5 consultas,3. R$ 56.01 - R$ 84
16594,65423402,02634816545,2023-06-29,2023-09-13,0.0,3.0,30.0,1.0,BA,PORTO SEGURO,...,1,F: 39-43,A: 0-6 meses,8.3 km,8337.0,17 mins,1039.0,5. <= 45 dias,2. 2 consultas,4. R$ 84.01 - R$ 140
16595,62807261,11163831441,2023-07-14,2023-09-28,0.0,2.0,15.0,0.0,PE,RECIFE,...,1,D: 29-33,A: 0-6 meses,5.9 km,5931.0,18 mins,1106.0,5. <= 45 dias,2. 2 consultas,4. R$ 84.01 - R$ 140
16596,62120292,12912946514,2022-06-22,2022-12-01,1.0,15.0,316.0,10.0,SE,ARACAJU,...,1,A: 0-18,C: 1 ano,3.8 km,3763.0,12 mins,724.0,2. 181-365 dias,3. 3 consultas,2. R$ 28.01 - R$ 56


In [ ]:
# MATERIALIZAR A BASE FINAL EM EXCEL

df_pac.to_excel('bd_paciente_geo.xlsx', index=False, sheet_name='base')

In [ ]:
query_titular = f"""
--BASE PACIENTES TITULARES
--SELECT count(*) from (
WITH grouped_pacientes AS (
    SELECT
        cr.id_paciente,
        cr.cpfpaciente,
        MIN(cr.datapagamento) AS min_dt_pg,
        MAX(cr.datapagamento) AS max_dt_pg,
        DATE_DIFF('year', MIN(cr.datapagamento), current_date) AS anos_utilizacao,
        DATE_DIFF('month', MIN(cr.datapagamento), current_date) AS meses_utilizacao,
        DATE_DIFF('day', MAX(cr.datapagamento), current_date) AS dias_ult_utilizacao,
        DATE_DIFF('month', MAX(cr.datapagamento), current_date) AS meses_ult_utilizacao,
        UPPER(pe.estado) AS estado,
    	UPPER(pe.cidade) AS cidade,
    	DATE_DIFF('year', pc.nascimento, current_date) AS idade,
    	sx.nomesexo AS sexo,
    	upper(concat(regexp_replace(regexp_replace(pe.logradouro, '[0-9]', ''), ', ', ''), ', ', pe.numero, ', ', pe.cidade, ', ', pe.estado)) endereco_paciente
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr 
    LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON pe.paciente_id = cr.id_paciente
	LEFT JOIN todos_data_lake_trusted_feegow.pacientes pc ON pc.id = cr.id_paciente
	LEFT JOIN todos_data_lake_trusted_feegow.sexo sx ON sx.id = pc.sexo
    WHERE 1=1
    	AND pc.sexo IN (1,2)
    	AND pc.sys_active = 1
	    AND pc.nascimento IS NOT NULL
		AND pe.logradouro IS NOT NULL
		AND pe.numero IS NOT NULL
		AND pe.cidade IS NOT NULL
	    --AND pc.email IS NOT NULL
	    --AND pc.email <> ''
		AND cr.cpfpaciente IS NOT NULL
		AND cr.cpfpaciente <> ''
		AND pc.nascimento IS NOT NULL
		AND pe.estado IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 
		'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
    GROUP BY
        cr.id_paciente,
        cr.cpfpaciente,
        pe.estado,
        pe.cidade,
        pe.bairro,
        pc.nascimento,
        sx.nomesexo,
        pe.logradouro,
        pe.numero,
        pe.cidade,
        pe.estado
),
consultas AS (
    SELECT
        cr.id_paciente,
        COUNT(*) AS qtd_consultas,
        SUM(cr.valor_pago) AS tt_consulta,
        AVG(cr.valor_pago) AS tm_consulta
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
    WHERE 1=1
    	AND cr.nomegrupo in ('Consultas', 'Sessão')
    GROUP BY
        cr.id_paciente
 ),
orcamento_nao_executado AS (
    SELECT
        pp.pacienteid AS id_paciente,
        COUNT(*) AS qtd_orc_nao_executado,
        SUM(pp.valor) AS tt_orc_nao_executado
    FROM todos_data_lake_trusted_feegow.propostas pp
    WHERE 1=1
        AND pp.staid NOT IN (5, 2)
    GROUP BY
        pp.pacienteid
 ), 
particular AS (
    SELECT
        cr.id_paciente,
        COUNT(*) AS qtd_particular
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
    WHERE 1=1
        AND cr.nome_tabela_particular = 'PARTICULAR*'
        AND cr.nomegrupo in ('Consultas', 'Sessão')
    GROUP BY
        cr.id_paciente
 ), 
exames_procedimentos AS (
    SELECT
        base.id_paciente,
        SUM(base.tt_exam_proc_data) tt_exam_proc,
        SUM(base.visitas) AS qtd_exam_proc
    FROM (
        SELECT
            cr.id_paciente,
            cr.datapagamento,
            SUM(cr.valor_pago) AS tt_exam_proc_data,
            1 AS visitas
        FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
        WHERE 1=1
        AND cr.nomegrupo IN ('Exames Laboratoriais', 'Procedimentos' , 'Exames de Imagem')
        GROUP BY
            cr.id_paciente, 
            cr.datapagamento
    ) base
    GROUP BY
        base.id_paciente
), 
canal AS (
	WITH canal_paciente AS (
	    SELECT
	        ag.id_paciente,
	        CASE WHEN ag.nm_canal = 'Clínica' THEN ag.nm_subcanal 
	        	 WHEN ag.nm_canal = 'App Cartão de Todos' THEN 'Aplicativo'
	        	 WHEN ag.nm_canal IN ('Site AmorSaúde', 'LabNine', 'Tutti', 'WhatsApp', 'Telefone', 'Chat') THEN 'Outros'
	        	 ELSE ag.nm_canal END nm_canal,
	        count(*) qtd_canais,
	        ROW_NUMBER() OVER(PARTITION BY ag.id_paciente ORDER BY max(ag.dt_agendamento) DESC, ag.nm_canal) AS rn
	    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
	    WHERE 1=1
	    	AND ag.nm_canal IS NOT NULL
	    	AND ag.tipoprocedimento = 'Consulta'
	    GROUP BY
	        ag.id_paciente,
	        ag.nm_canal,
	        ag.nm_subcanal
    )
SELECT 
    cp.id_paciente,
    COALESCE(cp.nm_canal, 'Não informado') AS nm_canal,
    cp.qtd_canais,
    CASE WHEN cp.qtd_canais > 1 THEN 1
    	 ELSE 0 END mudou_canal
FROM canal_paciente cp
WHERE cp.rn = 1
),
localidade AS (
	WITH regional_unidade_paciente AS (
	    SELECT
	        cr.id_paciente,
	        cr.regional,
	        cr.id_unidade,
	        cr.nome_unidade,
	        ROW_NUMBER() OVER(PARTITION BY cr.id_paciente ORDER BY COUNT(*) DESC, cr.regional) AS rn
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    --LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON 
	    	--pe.paciente_id = cr.id_paciente
	    WHERE 1=1
	    	AND cr.regional IS NOT NULL
	    	AND cr.id_unidade <> 19774
	    	AND cr.nome_unidade IS NOT NULL
	    GROUP BY
	        cr.id_paciente,
	        cr.regional,
	        cr.id_unidade,
	        cr.nome_unidade
    )
SELECT 
    rup.id_paciente,
    rup.id_unidade,
    rup.nome_unidade AS unidade,
    upper(concat(COALESCE(un.endereco, ' '), ', ', COALESCE(un.numero, ' '), ', ', COALESCE(un.cidade, ' '), ', ', COALESCE(un.estado, ' '))) AS endereco_unidade,
    upper(un.cidade) AS cidade_clinica,
    COALESCE(rup.regional, 'Não informado') AS regional
FROM regional_unidade_paciente rup
LEFT JOIN todos_data_lake_trusted_feegow.unidades un ON rup.id_unidade = un.id
WHERE rup.rn = 1
),
especialidade_mais_utilizada AS (
	WITH especialidade_paciente AS (
	    SELECT
	        ag.id_paciente,
	        ag.nm_especialidade,
	        ROW_NUMBER() OVER(PARTITION BY ag.id_paciente ORDER BY COUNT(*) DESC, ag.nm_especialidade) AS rn
	    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
	    WHERE 1=1
	    	AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
	    	AND ag.nm_especialidade IS NOT NULL 
	    GROUP BY
	        ag.id_paciente,
	        ag.nm_especialidade
    )
SELECT 
    ep.id_paciente,
    COALESCE(ep.nm_especialidade, 'Não informado') nm_especialidade
FROM especialidade_paciente ep
WHERE ep.rn = 1
),
tabela_mais_utilizada AS (
	WITH tabela_paciente AS (
	    SELECT
	    	bd.id_paciente,
	    	bd.tabela,
	    	ROW_NUMBER() OVER(PARTITION BY bd.id_paciente ORDER BY COUNT(*) DESC, bd.tabela) AS rn
	    FROM (SELECT
	        cr.id_paciente,
	        CASE WHEN cr.nome_tabela_particular = 'Cartão de TODOS*' THEN 'Cartão de TODOS'
             WHEN cr.nome_tabela_particular = 'PARTICULAR*' THEN 'Particular'
             ELSE 'Outros' END AS tabela
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    WHERE 1=1
	    	AND cr.nomegrupo in ('Consultas', 'Sessão')
	    GROUP BY
	        cr.id_paciente,
	        cr.nome_tabela_particular
    		 ) bd
    	GROUP BY
    		bd.id_paciente,
    		bd.tabela
    )
SELECT 
    tp.id_paciente,
    COALESCE(tp.tabela, 'Não informado') tabela
FROM tabela_paciente tp
WHERE tp.rn = 1
),
filiado_titular AS (
	SELECT 
		fl.matricula,
		fl.cpf,
		concat(upper(substring(fl.sexo, 1, 1)),
			   lower(substring(fl.sexo, 2))) sexo_fl,
		concat(upper(substring(fl.estado_civil, 1, 1)),
			   lower(substring(fl.estado_civil, 2))) estado_civil,
			   upper(fl.uf) uf,
		fl. idade idade_fl,
		fl.dependentes,
		fl.flag_desfiliado,
		fl.flag_refiliado,
		fl.status_atual_desc,
		fl.faixa_etaria faixa_etaria_fl,
		fl.dt_filiacao,
		fl.data_ultima_desfiliacao,
		fl.data_ultima_refiliacao,
		CASE
			WHEN fl.status_atual_desc = 'Ativo' THEN date_diff('month',
			fl.dt_filiacao,
			current_date)
			WHEN fl.status_atual_desc = 'Desfiliado' THEN date_diff('month',
			fl.dt_filiacao,
			fl.data_ultima_desfiliacao)
			ELSE NULL
		END AS meses_filiacao,
		upper(fl.prvd_tipo_venda_promotor) prvd_tipo_venda_promotor
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	WHERE
		1 = 1
		AND fl.flag_titular = 1
		AND fl.cpf IS NOT NULL
		AND fl.cpf NOT IN ('99199299394')
		AND fl.status_atual IN (0, 1)
),
base AS (SELECT DISTINCT
    gp.*,
    ft.*,
    COALESCE(cst.qtd_consultas, 0) AS qtd_consultas,
    COALESCE(expc.qtd_exam_proc, 0) AS qtd_exam_proc, 
    COALESCE(ptcl.qtd_particular, 0) AS qtd_particular,
    COALESCE(cst.tt_consulta, 0) AS tt_consulta,
    COALESCE(cst.tm_consulta, 0) AS tm_consulta,
    COALESCE(expc.tt_exam_proc, 0) AS tt_exam_proc,
    COALESCE(one.qtd_orc_nao_executado, 0) AS qtd_orc_nao_executado,
    COALESCE(one.tt_orc_nao_executado, 0) AS tt_orc_nao_executado,
    CASE WHEN COALESCE(cst.qtd_consultas, 0) <> 0 THEN (COALESCE(cst.tt_consulta, 0) + COALESCE(expc.tt_exam_proc, 0)) / cst.qtd_consultas
         ELSE NULL END AS tm_utilizacao,
    CASE WHEN cst.qtd_consultas = ptcl.qtd_particular AND cst.qtd_consultas <> 0 THEN 1
         ELSE 0 END AS pac_particular,
    ca.nm_canal AS ultimo_canal,
    ca.qtd_canais,
    ca.mudou_canal,
    loc.regional,
    emu.nm_especialidade,
    tmu.tabela,
    loc.unidade,
    loc.endereco_unidade,
    loc.cidade_clinica
FROM grouped_pacientes gp
LEFT JOIN consultas cst ON cst.id_paciente = gp.id_paciente
LEFT JOIN particular ptcl ON ptcl.id_paciente = gp.id_paciente
LEFT JOIN exames_procedimentos expc ON expc.id_paciente = gp.id_paciente
LEFT JOIN orcamento_nao_executado one ON one.id_paciente = gp.id_paciente
LEFT JOIN canal ca ON ca.id_paciente = gp.id_paciente
LEFT JOIN localidade loc ON loc.id_paciente = gp.id_paciente
LEFT JOIN especialidade_mais_utilizada emu ON emu.id_paciente = gp.id_paciente
LEFT JOIN tabela_mais_utilizada tmu ON tmu.id_paciente = gp.id_paciente
JOIN filiado_titular ft ON ft.cpf = gp.cpfpaciente
WHERE 1=1
	AND gp.idade BETWEEN 0 AND 100),
base_score AS (
SELECT 
	*,
    CASE WHEN bd.dias_ult_utilizacao > 365 THEN 1
         WHEN bd.dias_ult_utilizacao BETWEEN 181 AND 365 THEN 2
         WHEN bd.dias_ult_utilizacao BETWEEN 91 AND 180 THEN 3
         WHEN bd.dias_ult_utilizacao BETWEEN 46 AND 90 THEN 4
         WHEN bd.dias_ult_utilizacao <= 45 THEN 5
         ELSE NULL END AS score_r,
    CASE WHEN bd.qtd_consultas = 1 THEN 1
         WHEN bd.qtd_consultas = 2 THEN 2
         WHEN bd.qtd_consultas = 3 THEN 3
         WHEN bd.qtd_consultas BETWEEN 4 AND 5 THEN 4
         WHEN bd.qtd_consultas > 5 THEN 5
         ELSE NULL END AS score_f,
    CASE WHEN bd.tm_utilizacao <= 28 THEN 1
         WHEN bd.tm_utilizacao BETWEEN 28.01 AND 56 THEN 2
         WHEN bd.tm_utilizacao BETWEEN 56.01 AND 84 THEN 3
         WHEN bd.tm_utilizacao BETWEEN 84.01 AND 140 THEN 4
         WHEN bd.tm_utilizacao > 140 THEN 5
         ELSE NULL END AS score_v	
FROM base bd),
co_h AS (
SELECT
*
FROM base_score bs
WHERE 1=1
	AND bs.regional = 'CO'
	AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{co_h}
),	
co_m AS (
SELECT
*
FROM base_score bs
WHERE 1=1
	AND bs.regional = 'CO'
	AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{co_m}
),	
spcav_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP CAV'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{spcav_h}
),
spcav_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP CAV'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{spcav_m}
),
spint_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP Interior'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{spint_h}
),
spint_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP Interior'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{spint_m}
),
sul_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Sul'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{sul_h}
),
sul_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Sul'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{sul_m}
),
rj_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'RJ'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{rj_h}
),
rj_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'RJ'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{rj_m}
),
norte_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Norte'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{norte_h}
),
norte_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Norte'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{norte_m}
),
ne2_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE2'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{ne2_h}
),
ne2_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE2'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{ne2_m}
),
ne1_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE1'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{ne1_h}
),
ne1_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE1'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{ne1_m}
),
mg_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'MG'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{mg_h}
),
mg_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'MG'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{mg_m}
),
es_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'ES'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{es_h}
),
es_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'ES'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{es_m}
)
SELECT * FROM (
    SELECT * FROM co_h
    UNION ALL
    SELECT * FROM co_m
    UNION ALL
    SELECT * FROM spcav_h
    UNION ALL
    SELECT * FROM spcav_m
    UNION ALL
    SELECT * FROM spint_h
    UNION ALL
    SELECT * FROM spint_m
    UNION ALL
    SELECT * FROM sul_h
    UNION ALL
    SELECT * FROM sul_m
    UNION ALL
    SELECT * FROM rj_h
    UNION ALL
    SELECT * FROM rj_m
    UNION ALL
    SELECT * FROM norte_h
    UNION ALL
    SELECT * FROM norte_m
    UNION ALL
    SELECT * FROM ne2_h
    UNION ALL
    SELECT * FROM ne2_m
    UNION ALL
    SELECT * FROM ne1_h
    UNION ALL
    SELECT * FROM ne1_m
    UNION ALL
    SELECT * FROM mg_h
    UNION ALL
    SELECT * FROM mg_m
    UNION ALL
    SELECT * FROM es_h
    UNION ALL
    SELECT * FROM es_m
)  amostra
--)
"""

In [ ]:
# MATERIALIZACAO DA AMOSTRA DE TITULAR EM DF

df_tit = execute_athena_query(query_titular)

In [ ]:
# TRATATIVAS NA BASE DE TITULAR

# REMOVER ACENTOS DAS CIDADES 
def remove_accents(text):
    return ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])

# Apply the function to your columns
df_tit['cidade'] = df_tit['cidade'].apply(remove_accents)
df_tit['cidade_clinica'] = df_tit['cidade_clinica'].apply(remove_accents)

# APLICAR REGRA DE MESMA CIDADE
df_tit['mesma_cidade'] = (df_tit['cidade'] == df_tit['cidade_clinica']).astype(int)

# APLICAR FAIXAS DE IDADE E UTILIZACAO
age_bins = [0, 19, 24, 29, 34, 39, 44, 49, 54, 59, float('inf')]
age_labels = ['A: 0-18', 'B: 19-23', 'C: 24-28', 'D: 29-33', 'E: 34-38', 'F: 39-43', 'G: 44-48', 'H: 49-53', 'I: 54-58', 'J: 59+']

# Add a new column 'faixa_etaria' based on 'idade' column
df_tit['faixa_etaria'] = pd.cut(df_tit['idade'], bins=age_bins, labels=age_labels, right=False)

# Define the age group bins and labels based on your classification
time_bins = [0, 7, 13, 25, 37, 49, 61, float('inf')]
time_labels = ['A: 0-6 meses', 'B: 7-12 meses', 'C: 1 ano', 'D: 2 anos', 'E: 3 anos', 'F: 4 anos', 'G: 5+ anos']

# Add a new column 'faixa_etaria' based on 'idade' column
df_tit['faixa_utilizacao'] = pd.cut(df_tit['meses_utilizacao'], bins=time_bins, labels=time_labels, right=False)

In [ ]:
# Função para traduzir as condições da regra em texto
def translate_recency_score(dias_ult_utilizacao):
    if dias_ult_utilizacao <= 45:
        return '5. <= 45 dias'
    elif 46 <= dias_ult_utilizacao <= 90:
        return '4. 46-90 dias'
    elif 91 <= dias_ult_utilizacao <= 180:
        return '3. 91-180 dias'
    elif 181 <= dias_ult_utilizacao <= 365:
        return '2. 181-365 dias'
    elif dias_ult_utilizacao > 365:
        return '1. > 365 dias'
    else:
        return 'N/A'

# Aplicar a função à coluna 'dias_ult_utilizacao' e criar a nova coluna 'score_r_text'
df_tit['score_recencia'] = df_tit['dias_ult_utilizacao'].apply(translate_recency_score)

import pandas as pd

# Função para traduzir as condições da regra em texto
def translate_frequency_score(qtd_consultas):
    if qtd_consultas == 1:
        return '1. 1 consulta'
    elif qtd_consultas == 2:
        return '2. 2 consultas'
    elif qtd_consultas == 3:
        return '3. 3 consultas'
    elif 4 <= qtd_consultas <= 5:
        return '4. 4-5 consultas'
    elif qtd_consultas > 5:
        return '5. > 5 consultas'
    else:
        return 'N/A'

# Aplicar a função à coluna 'qtd_consultas' e 'tm_utilizacao' e criar a nova coluna 'score_f_text'
df_tit['score_frequencia'] = df_tit['qtd_consultas'].map(lambda x: translate_frequency_score(x))

# Função para traduzir as condições da regra em texto
def translate_value_score(tm_utilizacao):
    if tm_utilizacao is None:
        return 'N/A'
    elif tm_utilizacao <= 28:
        return '1. <= R$ 28'
    elif 28.01 <= tm_utilizacao <= 56:
        return '2. R$ 28.01 - R$ 56'
    elif 56.01 <= tm_utilizacao <= 84:
        return '3. R$ 56.01 - R$ 84'
    elif 84.01 <= tm_utilizacao <= 140:
        return '4. R$ 84.01 - R$ 140'
    elif tm_utilizacao > 140:
        return '5. > R$ 140'
    else:
        return 'N/A'

# Aplicar a função à coluna 'tm_utilizacao' e criar a nova coluna 'score_v_text'
df_tit['score_valor'] = df_tit['tm_utilizacao'].map(lambda x: translate_value_score(x))


In [ ]:
# Iterate through the DataFrame
for index, row in df_tit.iterrows():
    source = row['endereco_paciente']
    destination = row['endereco_unidade']

    try:
        if source and destination:
            direction_result = gmaps_client.directions(source, destination)

            # Extract the distance and duration values
            if direction_result:
                legs = direction_result[0]['legs'][0]
                distance_text = legs['distance']['text']
                distance_value = legs['distance']['value']
                duration_text = legs['duration']['text']
                duration_value = legs['duration']['value']

                # You can then use these values as needed, for example, store them in your DataFrame
                df_tit.at[index, 'distance_text'] = distance_text
                df_tit.at[index, 'distance_value'] = distance_value
                df_tit.at[index, 'duration_text'] = duration_text
                df_tit.at[index, 'duration_value'] = duration_value

    except ApiError as e:
        if "NOT_FOUND" in str(e):
            current_time = time.strftime('%Y-%m-%d %H:%M:%S')
            # Handle the "NOT_FOUND" error (e.g., skip to the next row)
            print(f"Skipping row {index} due to 'NOT_FOUND' error at {current_time}.")
            continue

Skipping row 27 due to 'NOT_FOUND' error at 2023-10-13 17:52:10.
Skipping row 818 due to 'NOT_FOUND' error at 2023-10-13 18:00:00.
Skipping row 1943 due to 'NOT_FOUND' error at 2023-10-13 18:10:56.
Skipping row 2667 due to 'NOT_FOUND' error at 2023-10-13 18:17:27.
Skipping row 2674 due to 'NOT_FOUND' error at 2023-10-13 18:17:31.
Skipping row 2675 due to 'NOT_FOUND' error at 2023-10-13 18:17:32.
Skipping row 2770 due to 'NOT_FOUND' error at 2023-10-13 18:18:27.
Skipping row 2886 due to 'NOT_FOUND' error at 2023-10-13 18:19:36.
Skipping row 3162 due to 'NOT_FOUND' error at 2023-10-13 18:22:16.
Skipping row 3455 due to 'NOT_FOUND' error at 2023-10-13 18:25:04.
Skipping row 3636 due to 'NOT_FOUND' error at 2023-10-13 18:26:47.
Skipping row 4099 due to 'NOT_FOUND' error at 2023-10-13 18:30:56.
Skipping row 4223 due to 'NOT_FOUND' error at 2023-10-13 18:31:55.
Skipping row 4829 due to 'NOT_FOUND' error at 2023-10-13 18:37:37.
Skipping row 5521 due to 'NOT_FOUND' error at 2023-10-13 18:43:35

In [ ]:
# MOSTRAR DF

df_tit

,id_paciente,cpfpaciente,min_dt_pg,max_dt_pg,anos_utilizacao,meses_utilizacao,dias_ult_utilizacao,meses_ult_utilizacao,estado,cidade,...,mesma_cidade,faixa_etaria,faixa_utilizacao,distance_text,distance_value,duration_text,duration_value,score_recencia,score_frequencia,score_valor
0,57941478,10117505706,2021-05-12,2021-05-25,2.0,29.0,871.0,28.0,ES,SERRA,...,1,F: 39-43,D: 2 anos,9.5 km,9476.0,17 mins,1016.0,1. > 365 dias,3. 3 consultas,3. R$ 56.01 - R$ 84
1,63632808,94861617715,2023-06-28,2023-09-01,0.0,3.0,42.0,1.0,ES,MANTENOPOLIS,...,0,J: 59+,A: 0-6 meses,131 km,130560.0,2 hours 8 mins,7653.0,5. <= 45 dias,N/A,N/A
2,10629581,38459680606,2021-09-18,2021-09-18,2.0,24.0,755.0,24.0,ES,SOORETAMA,...,0,J: 59+,C: 1 ano,23.4 km,23386.0,30 mins,1799.0,1. > 365 dias,1. 1 consulta,3. R$ 56.01 - R$ 84
3,59952024,13963128704,2021-11-24,2023-04-25,1.0,22.0,171.0,5.0,ES,SERRA,...,1,D: 29-33,C: 1 ano,4.0 km,4041.0,8 mins,488.0,3. 91-180 dias,4. 4-5 consultas,2. R$ 28.01 - R$ 56
4,66217152,10367601702,2023-07-04,2023-07-20,0.0,3.0,85.0,2.0,ES,VILA VELHA,...,0,G: 44-48,A: 0-6 meses,9.2 km,9178.0,18 mins,1105.0,4. 46-90 dias,2. 2 consultas,4. R$ 84.01 - R$ 140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16593,61194311,05376344878,2022-04-05,2023-06-29,1.0,18.0,106.0,3.0,SP,SAO PAULO,...,1,J: 59+,C: 1 ano,10.5 km,10519.0,21 mins,1282.0,3. 91-180 dias,4. 4-5 consultas,4. R$ 84.01 - R$ 140
16594,60594619,41177519801,2022-01-31,2022-01-31,1.0,20.0,620.0,20.0,SP,TABOAO DA SERRA,...,1,D: 29-33,C: 1 ano,3.8 km,3790.0,7 mins,390.0,1. > 365 dias,1. 1 consulta,1. <= R$ 28
16595,11446483,07522423856,2021-01-26,2023-07-26,2.0,32.0,79.0,2.0,SP,ITAPEVI,...,1,I: 54-58,D: 2 anos,10.4 km,10386.0,21 mins,1248.0,4. 46-90 dias,5. > 5 consultas,5. > R$ 140
16596,62852808,34951461803,2022-09-01,2022-10-25,1.0,13.0,353.0,11.0,SP,TABOAO DA SERRA,...,0,E: 34-38,C: 1 ano,10.0 km,9990.0,14 mins,818.0,2. 181-365 dias,2. 2 consultas,4. R$ 84.01 - R$ 140


In [ ]:
# MATERIALIZAR A BASE FINAL EM EXCEL

df_tit.to_excel('bd_titular_geo.xlsx', index=False, sheet_name='base')

In [ ]:
query_dependente = f"""
--BASE PACIENTES DEPENDENTES
--SELECT count(*) from (
WITH grouped_pacientes AS (
    SELECT
        cr.id_paciente,
        cr.cpfpaciente,
        MIN(cr.datapagamento) AS min_dt_pg,
        MAX(cr.datapagamento) AS max_dt_pg,
        DATE_DIFF('year', MIN(cr.datapagamento), current_date) AS anos_utilizacao,
        DATE_DIFF('month', MIN(cr.datapagamento), current_date) AS meses_utilizacao,
        DATE_DIFF('day', MAX(cr.datapagamento), current_date) AS dias_ult_utilizacao,
        DATE_DIFF('month', MAX(cr.datapagamento), current_date) AS meses_ult_utilizacao,
        UPPER(pe.estado) AS estado,
    	UPPER(pe.cidade) AS cidade,
    	DATE_DIFF('year', pc.nascimento, current_date) AS idade,
    	sx.nomesexo AS sexo,
    	upper(concat(regexp_replace(regexp_replace(pe.logradouro, '[0-9]', ''), ', ', ''), ', ', pe.numero, ', ', pe.cidade, ', ', pe.estado)) endereco_paciente
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr 
    LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON pe.paciente_id = cr.id_paciente
	LEFT JOIN todos_data_lake_trusted_feegow.pacientes pc ON pc.id = cr.id_paciente
	LEFT JOIN todos_data_lake_trusted_feegow.sexo sx ON sx.id = pc.sexo
    WHERE 1=1
    	AND pc.sexo IN (1,2)
    	AND pc.sys_active = 1
	    AND pc.nascimento IS NOT NULL
		AND pe.logradouro IS NOT NULL
		AND pe.numero IS NOT NULL
		AND pe.cidade IS NOT NULL
	    --AND pc.email IS NOT NULL
	    --AND pc.email <> ''
		AND cr.cpfpaciente IS NOT NULL
		AND cr.cpfpaciente <> ''
		AND pc.nascimento IS NOT NULL
		AND pe.estado IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 
		'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
    GROUP BY
        cr.id_paciente,
        cr.cpfpaciente,
        pe.estado,
        pe.cidade,
        pe.bairro,
        pc.nascimento,
        sx.nomesexo,
        pe.logradouro,
        pe.numero,
        pe.cidade,
        pe.estado
),
consultas AS (
    SELECT
        cr.id_paciente,
        COUNT(*) AS qtd_consultas,
        SUM(cr.valor_pago) AS tt_consulta,
        AVG(cr.valor_pago) AS tm_consulta
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
    WHERE 1=1
    	AND cr.nomegrupo in ('Consultas', 'Sessão')
    GROUP BY
        cr.id_paciente
 ),
orcamento_nao_executado AS (
    SELECT
        pp.pacienteid AS id_paciente,
        COUNT(*) AS qtd_orc_nao_executado,
        SUM(pp.valor) AS tt_orc_nao_executado
    FROM todos_data_lake_trusted_feegow.propostas pp
    WHERE 1=1
        AND pp.staid NOT IN (5, 2)
    GROUP BY
        pp.pacienteid
 ), 
particular AS (
    SELECT
        cr.id_paciente,
        COUNT(*) AS qtd_particular
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
    WHERE 1=1
        AND cr.nome_tabela_particular = 'PARTICULAR*'
        AND cr.nomegrupo in ('Consultas', 'Sessão')
    GROUP BY
        cr.id_paciente
 ), 
exames_procedimentos AS (
    SELECT
        base.id_paciente,
        SUM(base.tt_exam_proc_data) tt_exam_proc,
        SUM(base.visitas) AS qtd_exam_proc
    FROM (
        SELECT
            cr.id_paciente,
            cr.datapagamento,
            SUM(cr.valor_pago) AS tt_exam_proc_data,
            1 AS visitas
        FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
        WHERE 1=1
        AND cr.nomegrupo IN ('Exames Laboratoriais', 'Procedimentos' , 'Exames de Imagem')
        GROUP BY
            cr.id_paciente, 
            cr.datapagamento
    ) base
    GROUP BY
        base.id_paciente
), 
canal AS (
	WITH canal_paciente AS (
	    SELECT
	        ag.id_paciente,
	        CASE WHEN ag.nm_canal = 'Clínica' THEN ag.nm_subcanal 
	        	 WHEN ag.nm_canal = 'App Cartão de Todos' THEN 'Aplicativo'
	        	 WHEN ag.nm_canal IN ('Site AmorSaúde', 'LabNine', 'Tutti', 'WhatsApp', 'Telefone', 'Chat') THEN 'Outros'
	        	 ELSE ag.nm_canal END nm_canal,
	        count(*) qtd_canais,
	        ROW_NUMBER() OVER(PARTITION BY ag.id_paciente ORDER BY max(ag.dt_agendamento) DESC, ag.nm_canal) AS rn
	    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
	    WHERE 1=1
	    	AND ag.nm_canal IS NOT NULL
	    	AND ag.tipoprocedimento = 'Consulta'
	    GROUP BY
	        ag.id_paciente,
	        ag.nm_canal,
	        ag.nm_subcanal
    )
SELECT 
    cp.id_paciente,
    COALESCE(cp.nm_canal, 'Não informado') AS nm_canal,
    cp.qtd_canais,
    CASE WHEN cp.qtd_canais > 1 THEN 1
    	 ELSE 0 END mudou_canal
FROM canal_paciente cp
WHERE cp.rn = 1
),
localidade AS (
	WITH regional_unidade_paciente AS (
	    SELECT
	        cr.id_paciente,
	        cr.regional,
	        cr.id_unidade,
	        cr.nome_unidade,
	        ROW_NUMBER() OVER(PARTITION BY cr.id_paciente ORDER BY COUNT(*) DESC, cr.regional) AS rn
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    --LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON 
	    	--pe.paciente_id = cr.id_paciente
	    WHERE 1=1
	    	AND cr.regional IS NOT NULL
	    	AND cr.id_unidade <> 19774
	    	AND cr.nome_unidade IS NOT NULL
	    GROUP BY
	        cr.id_paciente,
	        cr.regional,
	        cr.id_unidade,
	        cr.nome_unidade
    )
SELECT 
    rup.id_paciente,
    rup.id_unidade,
    rup.nome_unidade AS unidade,
    upper(concat(COALESCE(un.endereco, ' '), ', ', COALESCE(un.numero, ' '), ', ', COALESCE(un.cidade, ' '), ', ', COALESCE(un.estado, ' '))) AS endereco_unidade,
    upper(un.cidade) AS cidade_clinica,
    COALESCE(rup.regional, 'Não informado') AS regional
FROM regional_unidade_paciente rup
LEFT JOIN todos_data_lake_trusted_feegow.unidades un ON rup.id_unidade = un.id
WHERE rup.rn = 1
),
especialidade_mais_utilizada AS (
	WITH especialidade_paciente AS (
	    SELECT
	        ag.id_paciente,
	        ag.nm_especialidade,
	        ROW_NUMBER() OVER(PARTITION BY ag.id_paciente ORDER BY COUNT(*) DESC, ag.nm_especialidade) AS rn
	    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
	    WHERE 1=1
	    	AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
	    	AND ag.nm_especialidade IS NOT NULL 
	    GROUP BY
	        ag.id_paciente,
	        ag.nm_especialidade
    )
SELECT 
    ep.id_paciente,
    COALESCE(ep.nm_especialidade, 'Não informado') nm_especialidade
FROM especialidade_paciente ep
WHERE ep.rn = 1
),
tabela_mais_utilizada AS (
	WITH tabela_paciente AS (
	    SELECT
	    	bd.id_paciente,
	    	bd.tabela,
	    	ROW_NUMBER() OVER(PARTITION BY bd.id_paciente ORDER BY COUNT(*) DESC, bd.tabela) AS rn
	    FROM (SELECT
	        cr.id_paciente,
	        CASE WHEN cr.nome_tabela_particular = 'Cartão de TODOS*' THEN 'Cartão de TODOS'
             WHEN cr.nome_tabela_particular = 'PARTICULAR*' THEN 'Particular'
             ELSE 'Outros' END AS tabela
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    WHERE 1=1
	    	AND cr.nomegrupo in ('Consultas', 'Sessão')
	    GROUP BY
	        cr.id_paciente,
	        cr.nome_tabela_particular
    		 ) bd
    	GROUP BY
    		bd.id_paciente,
    		bd.tabela
    )
SELECT 
    tp.id_paciente,
    COALESCE(tp.tabela, 'Não informado') tabela
FROM tabela_paciente tp
WHERE tp.rn = 1
),
filiado_dependente AS (
	SELECT 
		fl.matricula,
		fl.cpf,
		concat(upper(substring(fl.sexo, 1, 1)),
			   lower(substring(fl.sexo, 2))) sexo_fl,
		concat(upper(substring(tit.sexo, 1, 1)),
			   lower(substring(tit.sexo, 2))) sexo_titular,
		concat(upper(substring(fl.estado_civil, 1, 1)),
			   lower(substring(fl.estado_civil, 2))) estado_civil,
			   upper(fl.uf) uf,
		fl. idade idade_fl,
		fl.flag_desfiliado,
		fl.flag_refiliado,
		fl.status_atual_desc,
		fl.dt_filiacao,
		fl.data_ultima_desfiliacao,
		fl.data_ultima_refiliacao,
		CASE
			WHEN fl.status_atual_desc = 'Ativo' THEN date_diff('month',
			fl.dt_filiacao,
			current_date)
			WHEN fl.status_atual_desc = 'Desfiliado' THEN date_diff('month',
			fl.dt_filiacao,
			fl.data_ultima_desfiliacao)
			ELSE NULL
		END AS meses_filiacao,
		upper(fl.prvd_tipo_venda_promotor) prvd_tipo_venda_promotor
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	LEFT JOIN pdgt_cartaodetodos_filiado.fl_filiado tit ON tit.id_filiado = fl.id_filiado_pai
	WHERE
		1 = 1
		AND fl.flag_titular = 0
		AND fl.cpf IS NOT NULL
		AND fl.cpf NOT IN ('99199299394')
		AND fl.status_atual IN (0, 1)
),
base AS (SELECT DISTINCT
    gp.*,
    fd.*,
    COALESCE(cst.qtd_consultas, 0) AS qtd_consultas,
    COALESCE(expc.qtd_exam_proc, 0) AS qtd_exam_proc, 
    COALESCE(ptcl.qtd_particular, 0) AS qtd_particular,
    COALESCE(cst.tt_consulta, 0) AS tt_consulta,
    COALESCE(cst.tm_consulta, 0) AS tm_consulta,
    COALESCE(expc.tt_exam_proc, 0) AS tt_exam_proc,
    COALESCE(one.qtd_orc_nao_executado, 0) AS qtd_orc_nao_executado,
    COALESCE(one.tt_orc_nao_executado, 0) AS tt_orc_nao_executado,
    CASE WHEN COALESCE(cst.qtd_consultas, 0) <> 0 THEN (COALESCE(cst.tt_consulta, 0) + COALESCE(expc.tt_exam_proc, 0)) / cst.qtd_consultas
         ELSE NULL END AS tm_utilizacao,
    CASE WHEN cst.qtd_consultas = ptcl.qtd_particular AND cst.qtd_consultas <> 0 THEN 1
         ELSE 0 END AS pac_particular,
    ca.nm_canal AS ultimo_canal,
    ca.qtd_canais,
    ca.mudou_canal,
    loc.regional,
    emu.nm_especialidade,
    tmu.tabela,
    loc.unidade,
    loc.endereco_unidade,
    loc.cidade_clinica
FROM grouped_pacientes gp
LEFT JOIN consultas cst ON cst.id_paciente = gp.id_paciente
LEFT JOIN particular ptcl ON ptcl.id_paciente = gp.id_paciente
LEFT JOIN exames_procedimentos expc ON expc.id_paciente = gp.id_paciente
LEFT JOIN orcamento_nao_executado one ON one.id_paciente = gp.id_paciente
LEFT JOIN canal ca ON ca.id_paciente = gp.id_paciente
LEFT JOIN localidade loc ON loc.id_paciente = gp.id_paciente
LEFT JOIN especialidade_mais_utilizada emu ON emu.id_paciente = gp.id_paciente
LEFT JOIN tabela_mais_utilizada tmu ON tmu.id_paciente = gp.id_paciente
JOIN filiado_dependente fd ON fd.cpf = gp.cpfpaciente
WHERE 1=1
	AND gp.idade BETWEEN 0 AND 100),
base_score AS (
SELECT 
	*,
    CASE WHEN bd.dias_ult_utilizacao > 365 THEN 1
         WHEN bd.dias_ult_utilizacao BETWEEN 181 AND 365 THEN 2
         WHEN bd.dias_ult_utilizacao BETWEEN 91 AND 180 THEN 3
         WHEN bd.dias_ult_utilizacao BETWEEN 46 AND 90 THEN 4
         WHEN bd.dias_ult_utilizacao <= 45 THEN 5
         ELSE NULL END AS score_r,
    CASE WHEN bd.qtd_consultas = 1 THEN 1
         WHEN bd.qtd_consultas = 2 THEN 2
         WHEN bd.qtd_consultas = 3 THEN 3
         WHEN bd.qtd_consultas BETWEEN 4 AND 5 THEN 4
         WHEN bd.qtd_consultas > 5 THEN 5
         ELSE NULL END AS score_f,
    CASE WHEN bd.tm_utilizacao <= 28 THEN 1
         WHEN bd.tm_utilizacao BETWEEN 28.01 AND 56 THEN 2
         WHEN bd.tm_utilizacao BETWEEN 56.01 AND 84 THEN 3
         WHEN bd.tm_utilizacao BETWEEN 84.01 AND 140 THEN 4
         WHEN bd.tm_utilizacao > 140 THEN 5
         ELSE NULL END AS score_v	
FROM base bd),
co_h AS (
SELECT
*
FROM base_score bs
WHERE 1=1
	AND bs.regional = 'CO'
	AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{co_h}
),	
co_m AS (
SELECT
*
FROM base_score bs
WHERE 1=1
	AND bs.regional = 'CO'
	AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{co_m}
),	
spcav_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP CAV'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{spcav_h}
),
spcav_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP CAV'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{spcav_m}
),
spint_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP Interior'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{spint_h}
),
spint_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP Interior'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{spint_m}
),
sul_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Sul'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{sul_h}
),
sul_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Sul'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{sul_m}
),
rj_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'RJ'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{rj_h}
),
rj_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'RJ'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{rj_m}
),
norte_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Norte'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{norte_h}
),
norte_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Norte'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{norte_m}
),
ne2_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE2'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{ne2_h}
),
ne2_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE2'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{ne2_m}
),
ne1_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE1'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{ne1_h}
),
ne1_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE1'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{ne1_m}
),
mg_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'MG'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{mg_h}
),
mg_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'MG'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{mg_m}
),
es_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'ES'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{es_h}
),
es_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'ES'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{es_m}
)
SELECT * FROM (
    SELECT * FROM co_h
    UNION ALL
    SELECT * FROM co_m
    UNION ALL
    SELECT * FROM spcav_h
    UNION ALL
    SELECT * FROM spcav_m
    UNION ALL
    SELECT * FROM spint_h
    UNION ALL
    SELECT * FROM spint_m
    UNION ALL
    SELECT * FROM sul_h
    UNION ALL
    SELECT * FROM sul_m
    UNION ALL
    SELECT * FROM rj_h
    UNION ALL
    SELECT * FROM rj_m
    UNION ALL
    SELECT * FROM norte_h
    UNION ALL
    SELECT * FROM norte_m
    UNION ALL
    SELECT * FROM ne2_h
    UNION ALL
    SELECT * FROM ne2_m
    UNION ALL
    SELECT * FROM ne1_h
    UNION ALL
    SELECT * FROM ne1_m
    UNION ALL
    SELECT * FROM mg_h
    UNION ALL
    SELECT * FROM mg_m
    UNION ALL
    SELECT * FROM es_h
    UNION ALL
    SELECT * FROM es_m
)  amostra
--)
"""

In [ ]:
# MATERIALIZACAO DA AMOSTRA DE DEPENDENTES EM DF

df_dep = execute_athena_query(query_dependente)

In [ ]:
# TRATATIVAS NA BASE DE PACIENTES

# REMOVER ACENTOS DAS CIDADES
def remove_accents(text):
    return ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])

# Apply the function to your columns
df_dep['cidade'] = df_dep['cidade'].apply(remove_accents)
df_dep['cidade_clinica'] = df_dep['cidade_clinica'].apply(remove_accents)

# APLICAR REGRA DE MESMA CIDADE
df_dep['mesma_cidade'] = (df_dep['cidade'] == df_dep['cidade_clinica']).astype(int)

# APLICAR FAIXAS DE IDADE E UTILIZACAO
age_bins = [0, 19, 24, 29, 34, 39, 44, 49, 54, 59, float('inf')]
age_labels = ['A: 0-18', 'B: 19-23', 'C: 24-28', 'D: 29-33', 'E: 34-38', 'F: 39-43', 'G: 44-48', 'H: 49-53', 'I: 54-58', 'J: 59+']

# Add a new column 'faixa_etaria' based on 'idade' column
df_dep['faixa_etaria'] = pd.cut(df_dep['idade'], bins=age_bins, labels=age_labels, right=False)

# Define the age group bins and labels based on your classification
time_bins = [0, 7, 13, 25, 37, 49, 61, float('inf')]
time_labels = ['A: 0-6 meses', 'B: 7-12 meses', 'C: 1 ano', 'D: 2 anos', 'E: 3 anos', 'F: 4 anos', 'G: 5+ anos']

# Add a new column 'faixa_etaria' based on 'idade' column
df_dep['faixa_utilizacao'] = pd.cut(df_dep['meses_utilizacao'], bins=time_bins, labels=time_labels, right=False)


In [ ]:
# Função para traduzir as condições da regra em texto
def translate_recency_score(dias_ult_utilizacao):
    if dias_ult_utilizacao <= 45:
        return '5. <= 45 dias'
    elif 46 <= dias_ult_utilizacao <= 90:
        return '4. 46-90 dias'
    elif 91 <= dias_ult_utilizacao <= 180:
        return '3. 91-180 dias'
    elif 181 <= dias_ult_utilizacao <= 365:
        return '2. 181-365 dias'
    elif dias_ult_utilizacao > 365:
        return '1. > 365 dias'
    else:
        return 'N/A'

# Aplicar a função à coluna 'dias_ult_utilizacao' e criar a nova coluna 'score_r_text'
df_dep['score_recencia'] = df_dep['dias_ult_utilizacao'].apply(translate_recency_score)

import pandas as pd

# Função para traduzir as condições da regra em texto
def translate_frequency_score(qtd_consultas):
    if qtd_consultas == 1:
        return '1. 1 consulta'
    elif qtd_consultas == 2:
        return '2. 2 consultas'
    elif qtd_consultas == 3:
        return '3. 3 consultas'
    elif 4 <= qtd_consultas <= 5:
        return '4. 4-5 consultas'
    elif qtd_consultas > 5:
        return '5. > 5 consultas'
    else:
        return 'N/A'

# Aplicar a função à coluna 'qtd_consultas' e 'tm_utilizacao' e criar a nova coluna 'score_f_text'
df_dep['score_frequencia'] = df_dep['qtd_consultas'].map(lambda x: translate_frequency_score(x))

# Função para traduzir as condições da regra em texto
def translate_value_score(tm_utilizacao):
    if tm_utilizacao is None:
        return 'N/A'
    elif tm_utilizacao <= 28:
        return '1. <= R$ 28'
    elif 28.01 <= tm_utilizacao <= 56:
        return '2. R$ 28.01 - R$ 56'
    elif 56.01 <= tm_utilizacao <= 84:
        return '3. R$ 56.01 - R$ 84'
    elif 84.01 <= tm_utilizacao <= 140:
        return '4. R$ 84.01 - R$ 140'
    elif tm_utilizacao > 140:
        return '5. > R$ 140'
    else:
        return 'N/A'

# Aplicar a função à coluna 'tm_utilizacao' e criar a nova coluna 'score_valor'
df_dep['score_valor'] = df_dep['tm_utilizacao'].map(lambda x: translate_value_score(x))


In [ ]:
# Iterate through the DataFrame
for index, row in df_dep.iterrows():
    source = row['endereco_paciente']
    destination = row['endereco_unidade']

    try:
        if source and destination:
            direction_result = gmaps_client.directions(source, destination)

            # Extract the distance and duration values
            if direction_result:
                legs = direction_result[0]['legs'][0]
                distance_text = legs['distance']['text']
                distance_value = legs['distance']['value']
                duration_text = legs['duration']['text']
                duration_value = legs['duration']['value']

                # You can then use these values as needed, for example, store them in your DataFrame
                df_dep.at[index, 'distance_text'] = distance_text
                df_dep.at[index, 'distance_value'] = distance_value
                df_dep.at[index, 'duration_text'] = duration_text
                df_dep.at[index, 'duration_value'] = duration_value

    except ApiError as e:
        if "NOT_FOUND" in str(e):
            current_time = time.strftime('%Y-%m-%d %H:%M:%S')
            # Handle the "NOT_FOUND" error (e.g., skip to the next row)
            print(f"Skipping row {index} due to 'NOT_FOUND' error at {current_time}.")
            continue


Skipping row 132 due to 'NOT_FOUND' error at 2023-10-14 06:59:23.
Skipping row 230 due to 'NOT_FOUND' error at 2023-10-14 06:59:51.
Skipping row 392 due to 'NOT_FOUND' error at 2023-10-14 07:00:37.
Skipping row 1215 due to 'NOT_FOUND' error at 2023-10-14 07:04:29.
Skipping row 2438 due to 'NOT_FOUND' error at 2023-10-14 07:10:21.
Skipping row 2540 due to 'NOT_FOUND' error at 2023-10-14 07:10:51.
Skipping row 3989 due to 'NOT_FOUND' error at 2023-10-14 07:19:23.
Skipping row 4745 due to 'NOT_FOUND' error at 2023-10-14 07:24:29.
Skipping row 5844 due to 'NOT_FOUND' error at 2023-10-14 07:31:49.
Skipping row 7283 due to 'NOT_FOUND' error at 2023-10-14 07:41:39.
Skipping row 7381 due to 'NOT_FOUND' error at 2023-10-14 07:42:16.
Skipping row 7384 due to 'NOT_FOUND' error at 2023-10-14 07:42:17.
Skipping row 7636 due to 'NOT_FOUND' error at 2023-10-14 07:43:49.
Skipping row 8780 due to 'NOT_FOUND' error at 2023-10-14 07:51:21.
Skipping row 9646 due to 'NOT_FOUND' error at 2023-10-14 07:57:02

In [ ]:
# MOSTRAR DF

df_dep

,id_paciente,cpfpaciente,min_dt_pg,max_dt_pg,anos_utilizacao,meses_utilizacao,dias_ult_utilizacao,meses_ult_utilizacao,estado,cidade,...,mesma_cidade,faixa_etaria,faixa_utilizacao,distance_text,distance_value,duration_text,duration_value,score_recencia,score_frequencia,score_valor
0,66310659,08260713500,2023-07-11,2023-07-19,0.0,3.0,86.0,2.0,BA,LAURO DE FREITAS,...,1,A: 0-18,A: 0-6 meses,1.6 km,1562.0,5 mins,307.0,4. 46-90 dias,2. 2 consultas,4. R$ 84.01 - R$ 140
1,66173217,08443425563,2023-07-07,2023-07-07,0.0,3.0,98.0,3.0,BA,SALVADOR,...,1,A: 0-18,A: 0-6 meses,3.1 km,3060.0,11 mins,636.0,3. 91-180 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
2,66263801,06397073432,2023-07-11,2023-07-11,0.0,3.0,94.0,3.0,PE,RECIFE,...,1,E: 34-38,A: 0-6 meses,10.7 km,10686.0,25 mins,1527.0,3. 91-180 dias,N/A,N/A
3,14376301,06505231560,2021-12-29,2021-12-29,1.0,21.0,653.0,21.0,BA,PRESIDENTE JANIO QUADROS,...,0,D: 29-33,C: 1 ano,123 km,122972.0,1 hour 54 mins,6813.0,1. > 365 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
4,13818169,11455980560,2021-02-12,2023-10-06,2.0,32.0,7.0,0.0,BA,SALVADOR,...,0,A: 0-18,D: 2 anos,21.4 km,21355.0,30 mins,1827.0,5. <= 45 dias,4. 4-5 consultas,5. > R$ 140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16593,61955170,03988773867,2022-06-07,2023-10-05,1.0,16.0,8.0,0.0,SP,SAO PAULO,...,1,J: 59+,C: 1 ano,6.4 km,6410.0,13 mins,808.0,5. <= 45 dias,5. > 5 consultas,5. > R$ 140
16594,60284351,53557959864,2022-01-10,2023-03-30,1.0,21.0,197.0,6.0,SP,SAO PAULO,...,1,A: 0-18,C: 1 ano,7.9 km,7928.0,20 mins,1187.0,2. 181-365 dias,2. 2 consultas,2. R$ 28.01 - R$ 56
16595,11276239,54840265801,2021-12-02,2023-09-30,1.0,22.0,13.0,0.0,SP,APARECIDA,...,0,B: 19-23,C: 1 ano,6.5 km,6502.0,14 mins,824.0,5. <= 45 dias,4. 4-5 consultas,2. R$ 28.01 - R$ 56
16596,18047663,54260913824,2023-09-16,2023-09-16,0.0,0.0,27.0,0.0,SP,SAO PAULO,...,1,B: 19-23,A: 0-6 meses,4.3 km,4325.0,14 mins,815.0,5. <= 45 dias,1. 1 consulta,3. R$ 56.01 - R$ 84


In [ ]:
# MATERIALIZAR A BASE FINAL EM EXCEL

df_dep.to_excel('bd_dependente_geo.xlsx', index=False, sheet_name='base')

In [ ]:
query_particular = f"""
--BASE PACIENTES PARTICULARES
--SELECT count(*) from (
WITH grouped_pacientes AS (
    SELECT
        cr.id_paciente,
        cr.cpfpaciente,
        MIN(cr.datapagamento) AS min_dt_pg,
        MAX(cr.datapagamento) AS max_dt_pg,
        DATE_DIFF('year', MIN(cr.datapagamento), current_date) AS anos_utilizacao,
        DATE_DIFF('month', MIN(cr.datapagamento), current_date) AS meses_utilizacao,
        DATE_DIFF('day', MAX(cr.datapagamento), current_date) AS dias_ult_utilizacao,
        DATE_DIFF('month', MAX(cr.datapagamento), current_date) AS meses_ult_utilizacao,
        UPPER(pe.estado) AS estado,
    	UPPER(pe.cidade) AS cidade,
    	DATE_DIFF('year', pc.nascimento, current_date) AS idade,
    	sx.nomesexo AS sexo,
    	upper(concat(regexp_replace(regexp_replace(pe.logradouro, '[0-9]', ''), ', ', ''), ', ', pe.numero, ', ', pe.cidade, ', ', pe.estado)) endereco_paciente
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr 
    LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON pe.paciente_id = cr.id_paciente
	LEFT JOIN todos_data_lake_trusted_feegow.pacientes pc ON pc.id = cr.id_paciente
	LEFT JOIN todos_data_lake_trusted_feegow.sexo sx ON sx.id = pc.sexo
    WHERE 1=1
    	AND pc.sexo IN (1,2)
    	AND pc.sys_active = 1
	    AND pc.nascimento IS NOT NULL
		AND pe.logradouro IS NOT NULL
		AND pe.numero IS NOT NULL
		AND pe.cidade IS NOT NULL
	    --AND pc.email IS NOT NULL
	    --AND pc.email <> ''
		AND cr.cpfpaciente IS NOT NULL
		AND cr.cpfpaciente <> ''
		AND pc.nascimento IS NOT NULL
		AND pe.estado IN ('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 
		'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO')
    GROUP BY
        cr.id_paciente,
        cr.cpfpaciente,
        pe.estado,
        pe.cidade,
        pe.bairro,
        pc.nascimento,
        sx.nomesexo,
        pe.logradouro,
        pe.numero,
        pe.cidade,
        pe.estado
),
consultas AS (
    SELECT
        cr.id_paciente,
        COUNT(*) AS qtd_consultas,
        SUM(cr.valor_pago) AS tt_consulta,
        AVG(cr.valor_pago) AS tm_consulta
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
    WHERE 1=1
    	AND cr.nomegrupo in ('Consultas', 'Sessão')
    GROUP BY
        cr.id_paciente
 ),
orcamento_nao_executado AS (
    SELECT
        pp.pacienteid AS id_paciente,
        COUNT(*) AS qtd_orc_nao_executado,
        SUM(pp.valor) AS tt_orc_nao_executado
    FROM todos_data_lake_trusted_feegow.propostas pp
    WHERE 1=1
        AND pp.staid NOT IN (5, 2)
    GROUP BY
        pp.pacienteid
 ), 
particular AS (
    SELECT
        cr.id_paciente,
        COUNT(*) AS qtd_particular
    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
    WHERE 1=1
        AND cr.nome_tabela_particular = 'PARTICULAR*'
        AND cr.nomegrupo in ('Consultas', 'Sessão')
    GROUP BY
        cr.id_paciente
 ), 
exames_procedimentos AS (
    SELECT
        base.id_paciente,
        SUM(base.tt_exam_proc_data) tt_exam_proc,
        SUM(base.visitas) AS qtd_exam_proc
    FROM (
        SELECT
            cr.id_paciente,
            cr.datapagamento,
            SUM(cr.valor_pago) AS tt_exam_proc_data,
            1 AS visitas
        FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
        WHERE 1=1
        AND cr.nomegrupo IN ('Exames Laboratoriais', 'Procedimentos' , 'Exames de Imagem')
        GROUP BY
            cr.id_paciente, 
            cr.datapagamento
    ) base
    GROUP BY
        base.id_paciente
), 
canal AS (
	WITH canal_paciente AS (
	    SELECT
	        ag.id_paciente,
	        CASE WHEN ag.nm_canal = 'Clínica' THEN ag.nm_subcanal 
	        	 WHEN ag.nm_canal = 'App Cartão de Todos' THEN 'Aplicativo'
	        	 WHEN ag.nm_canal IN ('Site AmorSaúde', 'LabNine', 'Tutti', 'WhatsApp', 'Telefone', 'Chat') THEN 'Outros'
	        	 ELSE ag.nm_canal END nm_canal,
	        count(*) qtd_canais,
	        ROW_NUMBER() OVER(PARTITION BY ag.id_paciente ORDER BY max(ag.dt_agendamento) DESC, ag.nm_canal) AS rn
	    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
	    WHERE 1=1
	    	AND ag.nm_canal IS NOT NULL
	    	AND ag.tipoprocedimento = 'Consulta'
	    GROUP BY
	        ag.id_paciente,
	        ag.nm_canal,
	        ag.nm_subcanal
    )
SELECT 
    cp.id_paciente,
    COALESCE(cp.nm_canal, 'Não informado') AS nm_canal,
    cp.qtd_canais,
    CASE WHEN cp.qtd_canais > 1 THEN 1
    	 ELSE 0 END mudou_canal
FROM canal_paciente cp
WHERE cp.rn = 1
),
localidade AS (
	WITH regional_unidade_paciente AS (
	    SELECT
	        cr.id_paciente,
	        cr.regional,
	        cr.id_unidade,
	        cr.nome_unidade,
	        ROW_NUMBER() OVER(PARTITION BY cr.id_paciente ORDER BY COUNT(*) DESC, cr.regional) AS rn
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    --LEFT JOIN todos_data_lake_trusted_feegow.paciente_endereco pe ON 
	    	--pe.paciente_id = cr.id_paciente
	    WHERE 1=1
	    	AND cr.regional IS NOT NULL
	    	AND cr.id_unidade <> 19774
	    	AND cr.nome_unidade IS NOT NULL
	    GROUP BY
	        cr.id_paciente,
	        cr.regional,
	        cr.id_unidade,
	        cr.nome_unidade
    )
SELECT 
    rup.id_paciente,
    rup.id_unidade,
    rup.nome_unidade AS unidade,
    upper(concat(COALESCE(un.endereco, ' '), ', ', COALESCE(un.numero, ' '), ', ', COALESCE(un.cidade, ' '), ', ', COALESCE(un.estado, ' '))) AS endereco_unidade,
    upper(un.cidade) AS cidade_clinica,
    COALESCE(rup.regional, 'Não informado') AS regional
FROM regional_unidade_paciente rup
LEFT JOIN todos_data_lake_trusted_feegow.unidades un ON rup.id_unidade = un.id
WHERE rup.rn = 1
),
especialidade_mais_utilizada AS (
	WITH especialidade_paciente AS (
	    SELECT
	        ag.id_paciente,
	        ag.nm_especialidade,
	        ROW_NUMBER() OVER(PARTITION BY ag.id_paciente ORDER BY COUNT(*) DESC, ag.nm_especialidade) AS rn
	    FROM pdgt_amorsaude_operacoes.fl_agendamentos ag
	    WHERE 1=1
	    	AND ag.id_status IN (2, 3, 4, 5, 33, 200, 201, 202, 203, 204, 205, 206, 207)
	    	AND ag.nm_especialidade IS NOT NULL 
	    GROUP BY
	        ag.id_paciente,
	        ag.nm_especialidade
    )
SELECT 
    ep.id_paciente,
    COALESCE(ep.nm_especialidade, 'Não informado') nm_especialidade
FROM especialidade_paciente ep
WHERE ep.rn = 1
),
tabela_mais_utilizada AS (
	WITH tabela_paciente AS (
	    SELECT
	    	bd.id_paciente,
	    	bd.tabela,
	    	ROW_NUMBER() OVER(PARTITION BY bd.id_paciente ORDER BY COUNT(*) DESC, bd.tabela) AS rn
	    FROM (SELECT
	        cr.id_paciente,
	        CASE WHEN cr.nome_tabela_particular = 'Cartão de TODOS*' THEN 'Cartão de TODOS'
             WHEN cr.nome_tabela_particular = 'PARTICULAR*' THEN 'Particular'
             ELSE 'Outros' END AS tabela
	    FROM pdgt_amorsaude_financeiro.fl_contas_a_receber cr
	    WHERE 1=1
	    	AND cr.nomegrupo in ('Consultas', 'Sessão')
	    GROUP BY
	        cr.id_paciente,
	        cr.nome_tabela_particular
    		 ) bd
    	GROUP BY
    		bd.id_paciente,
    		bd.tabela
    )
SELECT 
    tp.id_paciente,
    COALESCE(tp.tabela, 'Não informado') tabela
FROM tabela_paciente tp
WHERE tp.rn = 1
),
filiado_titular AS (
	SELECT 
		fl.matricula,
		fl.cpf,
		concat(upper(substring(fl.sexo, 1, 1)),
			   lower(substring(fl.sexo, 2))) sexo_fl,
		concat(upper(substring(fl.estado_civil, 1, 1)),
			   lower(substring(fl.estado_civil, 2))) estado_civil,
			   upper(fl.uf) uf,
		fl. idade idade_fl,
		fl.dependentes,
		fl.flag_desfiliado,
		fl.flag_refiliado,
		fl.status_atual_desc,
		fl.faixa_etaria faixa_etaria_fl,
		fl.dt_filiacao,
		fl.data_ultima_desfiliacao,
		fl.data_ultima_refiliacao,
		CASE
			WHEN fl.status_atual_desc = 'Ativo' THEN date_diff('month',
			fl.dt_filiacao,
			current_date)
			WHEN fl.status_atual_desc = 'Desfiliado' THEN date_diff('month',
			fl.dt_filiacao,
			fl.data_ultima_desfiliacao)
			ELSE NULL
		END AS meses_filiacao,
		upper(fl.prvd_tipo_venda_promotor) prvd_tipo_venda_promotor
	FROM
		pdgt_cartaodetodos_filiado.fl_filiado fl
	WHERE
		1 = 1
		--AND fl.flag_titular = 1
		AND fl.cpf NOT IN ('99199299394')
		AND fl.cpf IS NOT NULL
		AND fl.status_atual IN (0, 1)
),
base AS (SELECT DISTINCT
    gp.*,
    COALESCE(cst.qtd_consultas, 0) AS qtd_consultas,
    COALESCE(expc.qtd_exam_proc, 0) AS qtd_exam_proc, 
    COALESCE(ptcl.qtd_particular, 0) AS qtd_particular,
    COALESCE(cst.tt_consulta, 0) AS tt_consulta,
    COALESCE(cst.tm_consulta, 0) AS tm_consulta,
    COALESCE(expc.tt_exam_proc, 0) AS tt_exam_proc,
    COALESCE(one.qtd_orc_nao_executado, 0) AS qtd_orc_nao_executado,
    COALESCE(one.tt_orc_nao_executado, 0) AS tt_orc_nao_executado,
    CASE WHEN COALESCE(cst.qtd_consultas, 0) <> 0 THEN (COALESCE(cst.tt_consulta, 0) + COALESCE(expc.tt_exam_proc, 0)) / cst.qtd_consultas
         ELSE NULL END AS tm_utilizacao,
    CASE WHEN cst.qtd_consultas = ptcl.qtd_particular AND cst.qtd_consultas <> 0 THEN 1
         ELSE 0 END AS pac_particular,
    ca.nm_canal AS ultimo_canal,
    ca.qtd_canais,
    ca.mudou_canal,
    loc.regional,
    emu.nm_especialidade,
    tmu.tabela,
    loc.unidade,
    loc.endereco_unidade,
    loc.cidade_clinica
FROM grouped_pacientes gp
LEFT JOIN consultas cst ON cst.id_paciente = gp.id_paciente
LEFT JOIN particular ptcl ON ptcl.id_paciente = gp.id_paciente
LEFT JOIN exames_procedimentos expc ON expc.id_paciente = gp.id_paciente
LEFT JOIN orcamento_nao_executado one ON one.id_paciente = gp.id_paciente
LEFT JOIN canal ca ON ca.id_paciente = gp.id_paciente
LEFT JOIN localidade loc ON loc.id_paciente = gp.id_paciente
LEFT JOIN especialidade_mais_utilizada emu ON emu.id_paciente = gp.id_paciente
LEFT JOIN tabela_mais_utilizada tmu ON tmu.id_paciente = gp.id_paciente
LEFT JOIN filiado_titular ft ON ft.cpf = gp.cpfpaciente
WHERE 1=1
	AND gp.idade BETWEEN 0 AND 100
	AND cst.qtd_consultas = ptcl.qtd_particular
	AND ft.cpf IS NULL),
base_score AS (
SELECT 
	*,
    CASE WHEN bd.dias_ult_utilizacao > 365 THEN 1
         WHEN bd.dias_ult_utilizacao BETWEEN 181 AND 365 THEN 2
         WHEN bd.dias_ult_utilizacao BETWEEN 91 AND 180 THEN 3
         WHEN bd.dias_ult_utilizacao BETWEEN 46 AND 90 THEN 4
         WHEN bd.dias_ult_utilizacao <= 45 THEN 5
         ELSE NULL END AS score_r,
    CASE WHEN bd.qtd_consultas = 1 THEN 1
         WHEN bd.qtd_consultas = 2 THEN 2
         WHEN bd.qtd_consultas = 3 THEN 3
         WHEN bd.qtd_consultas BETWEEN 4 AND 5 THEN 4
         WHEN bd.qtd_consultas > 5 THEN 5
         ELSE NULL END AS score_f,
    CASE WHEN bd.tm_utilizacao <= 28 THEN 1
         WHEN bd.tm_utilizacao BETWEEN 28.01 AND 56 THEN 2
         WHEN bd.tm_utilizacao BETWEEN 56.01 AND 84 THEN 3
         WHEN bd.tm_utilizacao BETWEEN 84.01 AND 140 THEN 4
         WHEN bd.tm_utilizacao > 140 THEN 5
         ELSE NULL END AS score_v	
FROM base bd),
co_h AS (
SELECT
*
FROM base_score bs
WHERE 1=1
	AND bs.regional = 'CO'
	AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{co_h_part}
),	
co_m AS (
SELECT
*
FROM base_score bs
WHERE 1=1
	AND bs.regional = 'CO'
	AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{co_m_part}
),	
spcav_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP CAV'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{spcav_h_part}
),
spcav_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP CAV'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{spcav_m_part}
),
spint_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP Interior'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{spint_h_part}
),
spint_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'SP Interior'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{spint_m_part}
),
sul_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Sul'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{sul_h_part}
),
sul_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Sul'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{sul_m_part}
),
rj_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'RJ'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{rj_h_part}
),
rj_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'RJ'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{rj_m_part}
),
norte_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Norte'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{norte_h_part}
),
norte_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'Norte'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{norte_m_part}
),
ne2_h AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE2'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{ne2_h_part}
),
ne2_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE2'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{ne2_m_part}
),
ne1_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE1'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{ne1_h_part}
),
ne1_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'NE1'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{ne1_m_part}
),
mg_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'MG'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{mg_h_part}
),
mg_m AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'MG'
    AND bs.sexo = 'Feminino'
ORDER BY
	RAND()
LIMIT
	{mg_m_part}
),
es_h AS ( 
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'ES'
    AND bs.sexo = 'Masculino'
ORDER BY
	RAND()
LIMIT
	{es_h_part}
),
es_m AS (
SELECT
    *
FROM base_score bs
WHERE bs.regional = 'ES'
    AND bs.sexo = 'Feminino'
    ORDER BY
	RAND()
LIMIT
	{es_m_part}
)
SELECT * FROM (
    SELECT * FROM co_h
    UNION ALL
    SELECT * FROM co_m
    UNION ALL
    SELECT * FROM spcav_h
    UNION ALL
    SELECT * FROM spcav_m
    UNION ALL
    SELECT * FROM spint_h
    UNION ALL
    SELECT * FROM spint_m
    UNION ALL
    SELECT * FROM sul_h
    UNION ALL
    SELECT * FROM sul_m
    UNION ALL
    SELECT * FROM rj_h
    UNION ALL
    SELECT * FROM rj_m
    UNION ALL
    SELECT * FROM norte_h
    UNION ALL
    SELECT * FROM norte_m
    UNION ALL
    SELECT * FROM ne2_h
    UNION ALL
    SELECT * FROM ne2_m
    UNION ALL
    SELECT * FROM ne1_h
    UNION ALL
    SELECT * FROM ne1_m
    UNION ALL
    SELECT * FROM mg_h
    UNION ALL
    SELECT * FROM mg_m
    UNION ALL
    SELECT * FROM es_h
    UNION ALL
    SELECT * FROM es_m
)  amostra
--)
"""

In [ ]:
# MATERIALIZACAO DA AMOSTRA DE PARTICULAR EM DF

df_part = execute_athena_query(query_particular)

In [ ]:
# TRATATIVAS NA BASE DE PACIENTES

# REMOVER ACENTOS DAS CIDADES
def remove_accents(text):
    return ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])

# Apply the function to your columns
df_part['cidade'] = df_part['cidade'].apply(remove_accents)
df_part['cidade_clinica'] = df_part['cidade_clinica'].apply(remove_accents)

# APLICAR REGRA DE MESMA CIDADE
df_part['mesma_cidade'] = (df_part['cidade'] == df_part['cidade_clinica']).astype(int)

# APLICAR FAIXAS DE IDADE E UTILIZACAO
age_bins = [0, 19, 24, 29, 34, 39, 44, 49, 54, 59, float('inf')]
age_labels = ['A: 0-18', 'B: 19-23', 'C: 24-28', 'D: 29-33', 'E: 34-38', 'F: 39-43', 'G: 44-48', 'H: 49-53', 'I: 54-58', 'J: 59+']

# Add a new column 'faixa_etaria' based on 'idade' column
df_part['faixa_etaria'] = pd.cut(df_part['idade'], bins=age_bins, labels=age_labels, right=False)

# Define the age group bins and labels based on your classification
time_bins = [0, 7, 13, 25, 37, 49, 61, float('inf')]
time_labels = ['A: 0-6 meses', 'B: 7-12 meses', 'C: 1 ano', 'D: 2 anos', 'E: 3 anos', 'F: 4 anos', 'G: 5+ anos']

# Add a new column 'faixa_etaria' based on 'idade' column
df_part['faixa_utilizacao'] = pd.cut(df_part['meses_utilizacao'], bins=time_bins, labels=time_labels, right=False)


In [ ]:
# Função para traduzir as condições da regra em texto
def translate_recency_score(dias_ult_utilizacao):
    if dias_ult_utilizacao <= 45:
        return '5. <= 45 dias'
    elif 46 <= dias_ult_utilizacao <= 90:
        return '4. 46-90 dias'
    elif 91 <= dias_ult_utilizacao <= 180:
        return '3. 91-180 dias'
    elif 181 <= dias_ult_utilizacao <= 365:
        return '2. 181-365 dias'
    elif dias_ult_utilizacao > 365:
        return '1. > 365 dias'
    else:
        return 'N/A'

# Aplicar a função à coluna 'dias_ult_utilizacao' e criar a nova coluna 'score_r_text'
df_part['score_recencia'] = df_part['dias_ult_utilizacao'].apply(translate_recency_score)

import pandas as pd

# Função para traduzir as condições da regra em texto
def translate_frequency_score(qtd_consultas):
    if qtd_consultas == 1:
        return '1. 1 consulta'
    elif qtd_consultas == 2:
        return '2. 2 consultas'
    elif qtd_consultas == 3:
        return '3. 3 consultas'
    elif 4 <= qtd_consultas <= 5:
        return '4. 4-5 consultas'
    elif qtd_consultas > 5:
        return '5. > 5 consultas'
    else:
        return 'N/A'

# Aplicar a função à coluna 'qtd_consultas' e 'tm_utilizacao' e criar a nova coluna 'score_f_text'
df_part['score_frequencia'] = df_part['qtd_consultas'].map(lambda x: translate_frequency_score(x))

# Função para traduzir as condições da regra em texto
def translate_value_score(tm_utilizacao):
    if tm_utilizacao is None:
        return 'N/A'
    elif tm_utilizacao <= 28:
        return '1. <= R$ 28'
    elif 28.01 <= tm_utilizacao <= 56:
        return '2. R$ 28.01 - R$ 56'
    elif 56.01 <= tm_utilizacao <= 84:
        return '3. R$ 56.01 - R$ 84'
    elif 84.01 <= tm_utilizacao <= 140:
        return '4. R$ 84.01 - R$ 140'
    elif tm_utilizacao > 140:
        return '5. > R$ 140'
    else:
        return 'N/A'

# Aplicar a função à coluna 'tm_utilizacao' e criar a nova coluna 'score_valor'
df_part['score_valor'] = df_part['tm_utilizacao'].map(lambda x: translate_value_score(x))


In [ ]:
# Iterate through the DataFrame
for index, row in df_part.iterrows():
    source = row['endereco_paciente']
    destination = row['endereco_unidade']

    try:
        if source and destination:
            direction_result = gmaps_client.directions(source, destination)

            # Extract the distance and duration values
            if direction_result:
                legs = direction_result[0]['legs'][0]
                distance_text = legs['distance']['text']
                distance_value = legs['distance']['value']
                duration_text = legs['duration']['text']
                duration_value = legs['duration']['value']

                # You can then use these values as needed, for example, store them in your DataFrame
                df_part.at[index, 'distance_text'] = distance_text
                df_part.at[index, 'distance_value'] = distance_value
                df_part.at[index, 'duration_text'] = duration_text
                df_part.at[index, 'duration_value'] = duration_value

    except ApiError as e:
        if "NOT_FOUND" in str(e):
            current_time = time.strftime('%Y-%m-%d %H:%M:%S')
            # Handle the "NOT_FOUND" error (e.g., skip to the next row)
            print(f"Skipping row {index} due to 'NOT_FOUND' error at {current_time}.")
            continue


Skipping row 318 due to 'NOT_FOUND' error at 2023-10-14 08:52:01.
Skipping row 1008 due to 'NOT_FOUND' error at 2023-10-14 08:56:23.
Skipping row 1155 due to 'NOT_FOUND' error at 2023-10-14 08:57:20.
Skipping row 1526 due to 'NOT_FOUND' error at 2023-10-14 08:59:39.
Skipping row 1815 due to 'NOT_FOUND' error at 2023-10-14 09:01:31.
Skipping row 2982 due to 'NOT_FOUND' error at 2023-10-14 09:08:49.
Skipping row 3005 due to 'NOT_FOUND' error at 2023-10-14 09:08:57.
Skipping row 3010 due to 'NOT_FOUND' error at 2023-10-14 09:08:59.
Skipping row 3094 due to 'NOT_FOUND' error at 2023-10-14 09:09:32.
Skipping row 3128 due to 'NOT_FOUND' error at 2023-10-14 09:09:46.


In [ ]:
# MOSTRAR DF

df_part

,id_paciente,cpfpaciente,min_dt_pg,max_dt_pg,anos_utilizacao,meses_utilizacao,dias_ult_utilizacao,meses_ult_utilizacao,estado,cidade,...,recencia,score_f_text,score_v_text,distance_text,distance_value,duration_text,duration_value,score_recencia,score_frequencia,score_valor
0,66368014,68611129334,2023-07-21,2023-07-21,0.0,2.0,85.0,2.0,CE,MARACANAU,...,46-90 dias,1 consulta,R$ 28.01 - R$ 56,5.2 km,5163.0,8 mins,463.0,4. 46-90 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
1,60826675,01368590462,2022-02-21,2022-02-21,1.0,19.0,600.0,19.0,PB,JOAO PESSOA,...,> 365 dias,1 consulta,R$ 28.01 - R$ 56,3.0 km,2999.0,9 mins,513.0,1. > 365 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
2,66574704,05308187363,2023-08-02,2023-08-09,0.0,2.0,66.0,2.0,CE,AQUIRAZ,...,46-90 dias,1 consulta,R$ 84.01 - R$ 140,5.3 km,5321.0,10 mins,578.0,4. 46-90 dias,1. 1 consulta,4. R$ 84.01 - R$ 140
3,14700995,71964410363,2021-01-14,2021-01-14,2.0,33.0,1003.0,33.0,PI,TERESINA,...,> 365 dias,1 consulta,R$ 84.01 - R$ 140,3.8 km,3807.0,12 mins,724.0,1. > 365 dias,1. 1 consulta,4. R$ 84.01 - R$ 140
4,64797482,32534710397,2023-03-02,2023-03-02,0.0,7.0,226.0,7.0,CE,MARACANAU,...,181-365 dias,1 consulta,R$ 28.01 - R$ 56,2.7 km,2743.0,7 mins,408.0,2. 181-365 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3156,64294233,89888367404,2023-01-23,2023-01-23,0.0,8.0,264.0,8.0,PE,PAULISTA,...,181-365 dias,1 consulta,R$ 28.01 - R$ 56,4.8 km,4775.0,12 mins,747.0,2. 181-365 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
3157,66512109,12177137438,2023-07-29,2023-07-29,0.0,2.0,77.0,2.0,PE,PAULISTA,...,46-90 dias,1 consulta,R$ 28.01 - R$ 56,6.3 km,6294.0,17 mins,996.0,4. 46-90 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
3158,64851033,85686597561,2023-03-20,2023-03-20,0.0,6.0,208.0,6.0,BA,SALVADOR,...,181-365 dias,1 consulta,R$ 28.01 - R$ 56,16.8 km,16826.0,22 mins,1317.0,2. 181-365 dias,1. 1 consulta,2. R$ 28.01 - R$ 56
3159,65223065,13153365539,2023-04-06,2023-04-06,0.0,6.0,191.0,6.0,BA,SALVADSOR,...,181-365 dias,1 consulta,R$ 28.01 - R$ 56,8.9 km,8949.0,14 mins,831.0,2. 181-365 dias,1. 1 consulta,2. R$ 28.01 - R$ 56


In [ ]:
# MATERIALIZAR A BASE FINAL EM EXCEL

df_part.to_excel('bd_particular_geo.xlsx', index=False, sheet_name='base')